## Set Up

In [ ]:
pip install arcgis

In [1]:
import arcpy
from datetime import datetime
import os
import sys
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import sqlalchemy as sa
import pandas as pd
from arcgis import GIS
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor, FeatureLayer
import pandas as pd
import numpy as np
import requests
from arcgis.gis import GIS
from arcgis.geometry import SpatialReference
#import geopandas as gpd to use spatial.reference stuff
gis = GIS()
# # Set Pandas display options to show all rows and columns
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

# in memory output file path
wk_memory = "memory" + "\\"


# set workspace and sde connections 
#working_folder = "C:\GIS"

#workspace      = "C:\GIS\Scratch.gdb"
#arcpy.env.workspace = "C:\GIS\Scratch.gdb"
#workspace ='F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'
#arcpy.env.workspace = 'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'

# network path to connection files
filePath = "F:\\GIS\\DB_CONNECT"

#path to GDB's to update and master data
master_path = "F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data\SEZ_Data.gdb"
SEZ_Master = os.path.join(master_path, "AssessmentUnits_Master")
#set workspace for connection to GDB
#workspace=master_path
# database file paths 
### SDE Collection New data collected is put into SDE.Survey under the indicator name
### SDE Vector is where the data will go 
sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
#sdeBase    = os.path.join(filePath, "SarahVector.sde")
#sdeCollect = os.path.join(filepath, "SarahCollect.sde")

# setup connection string???
#connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde;UID=sde;PWD=staff"
#connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
#engine = create_engine(connection_url)

#with engine.begin() as sdeConnect:
 #   erosiondf      = pd.read_sql("SELECT * FROM sde.SDE.Stream_Erosion", sdeConnect)

# local variables sdata is starting data and f data is finishing datatables
ffdata = os.path.join(sdeBase, "sde.SDE.Monitoring")
sdata = os.path.join(sdeCollect, "sde.SDE.Survey")
fdata = os.path.join(master_path, "AssessmentUnits_Master")

##Tables we get the data from in Collect 2010-2022 globalids don'tmatch
sezsurveytable = os.path.join(sdata, "sde.SDE.sez_survey")
erosiondata = os.path.join(sdata, "sde.SDE.Stream_Erosion")
incisiondata = os.path.join(sdata, "sde.SDE.sez_channel_incision")
invasivedata = os.path.join(sdata, "sde.SDE.sez_invasive_plant")
headcutdata = os.path.join(sdata, "sde.SDE.sez_stream_headcut")

#make this a spatial df
streamdata = os.path.join(ffdata, "sde.SDE.Stream")


#Staging Tables currently living in SEZ_Data.GDB
stage_bank_stability = os.path.join(master_path, "bank_stability") 
stage_All_SEZ_Scores = os.path.join(master_path, "All_SEZ_Scores")
stage_biotic_integrity = os.path.join(master_path, "biotic_integrity")
stage_headcuts = os.path.join(master_path, "headcuts_table")
stage_incision = os.path.join(master_path, "incision")
stage_invasives = os.path.join(master_path, "invasives")
stage_vegetation = os.path.join(master_path, "vegetation_vigor")
stage_conifer = os.path.join(master_path, "conifer_encroachment")
stage_aquatic = os.path.join(master_path, "aquatic_organism_passage_table")
stage_ditches = os.path.join(master_path, "ditches")
stage_habitat = os.path.join(master_path, "habitat_fragmentation")
#Final table to append to
#finalSEZtable = os.path.join(ffdata, "sde.SDE.SEZ_Assessment_Unit")
#finalSEZtable = os.path.join(master_path, "AssessmentUnits_Master")


# network path to connection files??????
#filePath = "C:\\GIS\\DB_CONNECT"
# network path to connection files
#filePath = r'F:\Research and Analysis\Workspace\Sarah'

#--------------------------------------------#
#Notes to self
#--------------------------------------------#
#F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data\SEZ_Data.gdb. 
###'Assessment_Unit_Master' has all data for SEZ
###“Bioassessment Sample Locations” has stream sites in SEZs

gdbworking_folder = "F:\GIS\GIS_DATA\Monitoring"
headcutgdbfolder = os.path.join(gdbworking_folder, "Stream_Headcut", "StreamHeadcut_Survey")
invasivegdbfolder = os.path.join(gdbworking_folder, "Invasive_Species", "Invasive_Species_Survey")
## GDB with Raw Data straight from S123 not in the original folder (that one is not edited)
headcut23gdb = os.path.join(headcutgdbfolder, "Stream_Headcut_Survey_2023.gdb")
headcut22gdb = os.path.join(headcutgdbfolder, "Stream_Headcut_Survey_2022.gdb")
headcut20gdb = os.path.join(headcutgdbfolder, "Stream_Headcut_Survey_2020.gdb")
headcut19gdb = os.path.join(headcutgdbfolder, "Stream_Headcut_Survey_2019.gdb")
#erosion23gdb = os.path.join(gdbworking_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2023.gdb")
#erosion22gdb = os.path.join(working_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2022.gdb")
#erosion20gdb = os.path.join(working_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2020.gdb")
#channelincision23gdb = os.path.join(working_folder,"Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2023.gdb")
#channelincision22gdb = os.path.join(working_folder, "Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2022.gdb")
#channelincision20gdb = os.path.join(working_folder, "Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2020.gdb")
invasiveplant23gdb= os.path.join(invasivegdbfolder,"Invasive_Species_Survey_2023.gdb")
invasiveplant22gdb= os.path.join(invasivegdbfolder,"Invasive_Species_Survey_2022.gdb")
invasiveplant20gdb= os.path.join(invasivegdbfolder,"Invasive_Species_Survey_2020.gdb")
invasiveplant19gdb= os.path.join(invasivegdbfolder,"Invasive_Species_Survey_2019.gdb")


#This is thelocatoin for the final SEZ GDB to be updated in the gdb on f drive in the AssessmentUnits Master (polygon) i believe
#FinalGDBtoupdate:F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data \SEZ_Data.gdb

#Location of USFS Invasive Species Data
#https://apps.fs.usda.gov/arcx/rest/services/EDW/EDW_InvasiveSpecies_01/MapServer

#Monitoring Dashboard location
#Finalsdelocation:f'Vector.SDE' Sde.Monitoring Sde. SEZ_Assessment_Unit

#Threshold Location? sde.tabular

In [2]:
def get_fs_data_spatial(fs_url):
    feature_layer = FeatureLayer(fs_url)
    query_result = feature_layer.query().sdf
    query_result.spatial.sr = 26910
    return query_result

SEZ_url = "https://maps.trpa.org/server/rest/services/SEZ_Assessment_Unit/FeatureServer/0"

dfSEZ = get_fs_data_spatial(SEZ_url)

### Use only if need to make SEZ ID dictionary

In [ ]:
#MAKE LOOKUP FOR SMALL ACRES SEZ ID AND FEATURE TYPE
# Filter Assessment_Unit_Name entries with more than one unique Acres value
grouped = dfSEZ.groupby('Assessment_Unit_Name')
filtered_df = grouped.filter(lambda x: x['Acres'].nunique() > 1)

# Determine the minimum Acres for each Assessment_Unit_Name
min_acres_df = filtered_df.groupby('Assessment_Unit_Name', as_index=False)['Acres'].min()

# Merge to get the corresponding SEZ_ID and SEZ_Type
result_df = pd.merge(min_acres_df, filtered_df, on=['Assessment_Unit_Name', 'Acres'])

# Select columns of interest for the CSV
columns_of_interest = ['Assessment_Unit_Name', 'SEZ_ID', 'SEZ_Type']

# Create a new DataFrame with selected columns
df_selected = result_df[columns_of_interest].copy()

# Specify the file path where you want to save the CSV file
csv_file_path = 'Small_Polygon_Lookup.csv'

# Save the selected data to CSV file
df_selected.to_csv(csv_file_path, index=False)

print(f"CSV file saved successfully to {csv_file_path}")

### Create SEZID and SEZType Lookup Dictionaries Small Polygone, Large Polygon, All Polygons

In [3]:
#Large Polygons or only polygon shapes lookup dictionary for Assessment Units with lerger values of acreage

# Step 1: Read the Excel file into a DataFrame
excel_data = pd.read_csv("F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Large_Polygon_Lookup.csv")  

#Define Empty look up dataframe
lookup_dict = {}

for index, row in excel_data.iterrows():
    lookup_dict[row['Assessment_Unit_Name']] = row['SEZ_ID']

# See dictionary where keys are Assessment Unit Names and values are SEZ IDs
print(lookup_dict)

#Small Polygon if there are two acres for an SEZ
# Step 1: Read the Excel file into a DataFrame
excel_data = pd.read_csv("F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Small_Polygon_Lookup.csv")  

#Define Empty look up dataframe
lookup_riverine = {}

for index, row in excel_data.iterrows():
    lookup_riverine[row['Assessment_Unit_Name']] = row['SEZ_ID']

# See dictionary where keys are Assessment Unit Names and values are SEZ IDs
print(lookup_riverine)

#All Polygons
# Step 1: Read the Excel file into a DataFrame
excel_data = pd.read_csv("F:\GIS\PROJECTS\ResearchAnalysis\SEZ\All_SEZID_Lookup.csv")  

#Define Empty look up dataframe
lookup_all = {}

for index, row in excel_data.iterrows():
    lookup_all[row['Assessment_Unit_Name']] = row['SEZ_ID']

# See dictionary where keys are Assessment Unit Names and values are SEZ IDs
print(lookup_all)


{'Angora Creek - tributary': 519, 'Angora Creek - upper': 520, 'Angora meadows - 1': 87, 'Angora meadows - 2': 90, 'Angora meadows - 3': 91, 'Angora meadows - 4': 143, 'Angora meadows - 5': 144, 'Angora meadows - 6': 142, 'Angora meadows - 7': 89, 'Angora meadows - 8': 88, 'Angora meadows - 9': 92, 'Angora meadows tributary - 1': 217, 'Angora meadows tributary - 2': 99, 'Angora meadows tributary - 3': 97, 'Angora meadows tributary - 4': 94, 'Angora meadows tributary - 5': 214, 'Angora meadows tributary - 6': 146, 'Angora meadows tributary - 7': 93, 'Angora meadows tributary - 8': 95, 'Angora meadows tributary - 9': 96, 'Angora tributary': 446, 'Antone meadows': 187, 'Baldwin marsh - 1': 160, 'Benwood meadows - 1': 129, 'Benwood meadows - 2': 131, 'Big Meadow - 1': 47, 'Big Meadow - 2': 48, 'Big Meadow - 3': 38, 'Big Meadow - 4': 39, 'Big Meadow - 5': 37, 'Big Meadow - 6': 40, 'Big Meadow Creek - lower': 521, 'Big Meadow Creek - upper': 491, 'Big Meadow Creek - upper 2': 522, 'Big meado

### Code for Grading each parameter

In [4]:
# Grading for each parameter 
#Defining Grade for Bank Stability based on Erosiondf[percent_unstable]
def categorize_erosion(Percent_Unstable):
    if pd.isna(Percent_Unstable):
        return np.nan
    elif 0 <= Percent_Unstable < 5:
        return 'A'
    elif 5 <= Percent_Unstable < 20:
        return 'B'
    elif 20 <= Percent_Unstable < 50:
        return 'C'
    else:
        return 'D'
    
#Scoring based off of grading - check this
def score_indicator(Rating):
    if pd.isna(Rating):
        return np.nan
    elif  Rating == 'A':
        return '12'
    elif Rating == 'B':
        return '9'
    elif Rating == 'C':
        return '6'
    else:
        return '3'

#Define Grade for Incision based off of incisino ratio

def categorize_incision(bankfull_ratio):
    if pd.isna(bankfull_ratio):
        return np.nan
    elif 0 <= bankfull_ratio < 1.2:
        return 'A'
    elif 1.2 <= bankfull_ratio < 1.6:
        return 'B'
    elif 1.6 <= bankfull_ratio < 2.1:
        return 'C'
    else:
        return 'D'

#Define Grade for Bioassessment Score
def categorize_csci(biotic_integrity):
     if pd.isna(biotic_integrity):
        return np.nan
     elif   biotic_integrity > 0.92:
        return 'A'
     elif 0.79 < biotic_integrity <= 0.92:
        return 'B'
     elif 0.62 < biotic_integrity <= 0.79:
        return 'C'
     else:
        return 'D'

#Define Priority List Level of Invasive Plant Species
    

# Process Data
def rate_invasive(priority):
    # Check if the SEZ has at least one large headcut
    if (priority[3] + priority[4] == 1):
        return 'B'  # Assign score D
    elif (priority[3] + priority[4] == 2) or priority[1] == 1 or priority[2] == 1:
        return 'C'  # Assign score C
    elif (priority[3] + priority[4] >= 3)or priority[1] >= 2 or priority[2] >= 2 or (priority[1] + priority[2] >= 2):  
       return 'D'  # Assign score B
    else:
        return 'A'  # Assign score A (no invasives of any priority are present)
     




#Define Size for Headcut based off of headcut size
##A = 0 headcut, B 1+small headcut
def categorize_headcut(headcutdepth):
    if pd.isnull(headcutdepth) or headcutdepth == 0:
        return 'None'
    elif 0.1 <= headcutdepth < 0.5:
        return 'small'
    elif 0.5 <= headcutdepth < 1:
        return 'medium'
    else:
        return 'large'

#define rating for headcut health per sez
def rate_headcut(row):
    # Check if the SEZ has at least one large headcut
    if row['large'] >= 1:
        return 'D'  # Assign score D
    elif row['medium'] >= 1:
        return 'C'  # Assign score C
    elif row['small'] >= 1:
       return 'B'  # Assign score B
    else:
        return 'A'  # Assign score A (no headcuts)


#define rating SEZ Rating
def rate_SEZ(percent):
    if 0 <= percent < .69:
        return 'D'
    elif .7 <= percent < .79:
        return 'C'
    elif .80 <= percent <= .89:
        return 'B'
    else:
        return 'A'
    
    #Define Grade for IPI Score - Used only for Stream HAbitat Condition
def categorize_phab(IPI):
     if   IPI >= 0.94:
        return 'A'
     elif 0.83 < IPI < 0.94:
        return 'B'
     elif 0.7 < IPI <= 0.83:
        return 'C'
     else:
        return 'D'



## Indicator Processing

### Stream Erosion 

In [ ]:
#----------------------------------------------------------------#
#Get Data
#----------------------------------------------------------------#
#Convert feature class in gdb to Pandas DataFrame this is for REST Service
erosionfields = ['Assessment_Unit_Name', 'Shape.STLength()', 'Bank_Type', 'Survey_Date']
#erosiondf = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(erosiondata, erosionfields), columns=erosionfields)

# Initialize an empty list to store data
data = []

 # Iterate over the rows in the feature class and extract data
with arcpy.da.SearchCursor(erosiondata, erosionfields) as cursor:
    for row in cursor:
        data.append(row)

# Convert the data to a Pandas DataFrame
erosiondf = pd.DataFrame(data, columns=erosionfields)

# Replace NaN values in 'Assessment_Unit_Name' column with 'Skylandia SEZ'
#erosiondf['Assessment_Unit_Name'] = erosiondf['Assessment_Unit_Name'].fillna('Skylandia SEZ')
# Replace specific values in 'Assessment_Unit_Name' column
erosiondf['Assessment_Unit_Name'] = erosiondf['Assessment_Unit_Name'].replace({'Blackwood Creek - upper 2': 'Blackwood Creek - Upper 2', 'Taylor Creek marsh - 1': 'Taylor Creek marsh'})


#Add SEZ_ID column using lookup dictionary
#erosiondf['SEZ_ID'] = erosiondf['Assessment_Unit_Name'].map(lambda x: lookup_dict.get(x, {}).get('SEZ_ID', None))

#This code is for the excel look up dictionary
erosiondf['SEZ_ID'] = erosiondf['Assessment_Unit_Name'].map(lookup_dict)


# Fill NaN values with a specific value, such as 0
erosiondf['SEZ_ID'] = erosiondf['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
erosiondf['SEZ_ID'] = erosiondf['SEZ_ID'].astype(int)

#erosiondf = erosiondf.merge(df, on='Assessment_Unit_Name', how='left')

#calculate year column 
erosiondf['Year'] = erosiondf['Survey_Date'].dt.year

# Replace 'both_banks' with 'Both Banks' in Bank_Type column
erosiondf['Bank_Type'] = erosiondf['Bank_Type'].replace(['both_banks', 'Both banks'], 'Both Banks' )
erosiondf['Bank_Type'] = erosiondf['Bank_Type'].replace(['one_bank', 'One bank'], 'One Bank')
erosiondf['Bank_Type'] = erosiondf['Bank_Type'].replace(['no_bank', 'No bank'], 'No Bank')

#----------------------------------------------------------------#
#Process Data
#----------------------------------------------------------------#

# Initialize variables
erosiondf['bank_multiplier'] = erosiondf['Bank_Type'].apply(lambda x: 2 if x == 'Both Banks' else (1 if x == 'One Bank' else 0))


# Calculate the product of 'Shape.STLength()' and 'bank_multiplier' to get the eroded banks per row
erosiondf['eroded_banks_per_row'] = erosiondf['Shape.STLength()'] * erosiondf['bank_multiplier']

# Group by Assessment_Unit_Name and year and sum the lengths of banks for each unit to get total banks assessed
erosiondf['banks_assessed_per_unit'] = erosiondf.groupby(['Assessment_Unit_Name', 'Year'])['Shape.STLength()'].transform('sum') * 2

# Group by Assessment_Unit_Name and sum the eroded banks per row for each unit
erosiondf['SEZ_total_eroded'] = erosiondf.groupby(['Assessment_Unit_Name', 'Year'])['eroded_banks_per_row'].transform('sum')

# Calculate percent unstable Bank Stability per Assessment Unit
erosiondf['Bank_Stability_Percent_Unstable'] = (erosiondf['SEZ_total_eroded'] / erosiondf['banks_assessed_per_unit']) * 100

#----------------------------------------------------------------#
#Grade, Score
#----------------------------------------------------------------#
erosiondf['Bank_Stability_Rating']=erosiondf['Bank_Stability_Percent_Unstable'].apply(categorize_erosion)
erosiondf['Bank_Stability_Score']= erosiondf['Bank_Stability_Rating'].apply(score_indicator)

erosiondf['Bank_Stability_Data_Source'] = 'TRPA'

erosiondf.head()

print(erosiondf)
#----------------------------------------------------------------#
#post ending dataframe to bank_stability called stage_bank_stability GDB location
#----------------------------------------------------------------#
# Define the name of the feature class
feature_class_name = 'bank_stability'

# Define the full path to the feature class
feature_class_path = stage_bank_stability 

#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'Year': 'Year',
    'Bank_Stability_Data_Source': 'Bank_Stability_Data_Source',
    'Bank_Stability_Percent_Unstable': 'Bank_Stability_Percent_Unstable',
    'Bank_Stability_Rating': 'Bank_Stability_Rating',
    'Bank_Stability_Score': 'Bank_Stability_Score',
    'SEZ_ID': 'SEZ_ID'
}

# Rename fields based on field mappings
bank_stabilitydf = erosiondf.rename(columns=field_mapping).drop(columns=[col for col in erosiondf.columns if col not in field_mapping])

readybankdf = bank_stabilitydf.groupby(['SEZ_ID', 'Year']).first().reset_index()

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

# Fix data type of Year so it writes to table
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y', errors='coerce')

# Setting the frequency to 'Y' for year
#readydf['Year'] = readydf['Year'].dt.to_period('Y')

print(readybankdf)

#Write dataframe to fc in SEZ_Data.GDB
# Convert DataFrame to a list of dictionaries
data = readybankdf.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(readybankdf.columns)

# Append data to existing table
#with arcpy.da.InsertCursor(stage_bank_stability, field_names) as cursor:
 #   for row in data:
  #      cursor.insertRow([row[field] for field in field_names])

#print("Data appended to table successfully.")


#Write dataframe to sde.collect.bank.stability eventually, current code write it to GDB in SEZ_Data.GDB
# Set environment workspace to your SDE connection file
#arcpy.env.workspace = master_path

# Convert DataFrame to Feature Class
#output_feature_class = "ErosionUpdate"  # Name for the output feature class
#output_fc_path = os.path.join(arcpy.env.workspace, output_feature_class)

# Assuming your DataFrame is already converted to a feature class
# Replace "path_to_your_feature_class" with the actual path to your feature class
#arcpy.conversion.TableToTable("path_to_your_feature_class", arcpy.env.workspace, output_feature_class)

# Overwrite Feature Class in SDE
# Replace "path_to_your_dataframe" with the actual path to your DataFrame
#arcpy.management.CopyFeatures("path_to_your_dataframe", output_fc_path)


#print(ready_df)



In [ ]:
# Convert DataFrame to CSV
#csv_path = os.path.join(working_folder, "Erosiondatamaster.csv")
#ready_df.to_csv(csv_path, index=False) 

### Channel Incision

In [24]:
#----------------------------------------------------------------#
#Get Data-- all the data we need is in sde.collect.sde.survey.sde.sez_survey
#----------------------------------------------------------------##Convert feature class in gdb to Pandas DataFrame this is for REST Service
incisionfields = ['Assessment_Unit_Name', 'incision_ratio', 'survey_date']
#incisiondf = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(erosiondata, erosionfields), columns=erosionfields)

# Initialize an empty list to store data
data = []

 # Iterate over the rows in the feature class and extract data
with arcpy.da.SearchCursor(sezsurveytable, incisionfields) as cursor:
    for row in cursor:
        data.append(row)

# Convert the data to a Pandas DataFrame
incisiondf = pd.DataFrame(data, columns=incisionfields)






#----------------------------------------------------------------#
#Add correct info to dataframe
#----------------------------------------------------------------#
#use this until we fix the domain
incisiondf['Assessment_Unit_Name'] = incisiondf['Assessment_Unit_Name'].replace({'Blackwood Creek - upper 2': 'Blackwood Creek - Upper 2', 'Taylor Creek marsh - 1': 'Taylor Creek marsh'})
# Create a new column 'SEZ ID'
#This code is for the excel look up dictionary
incisiondf['SEZ_ID'] = incisiondf['Assessment_Unit_Name'].map(lookup_dict)


# Fill NaN values with a specific value, such as 0
incisiondf['SEZ_ID'] = incisiondf['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
incisiondf['SEZ_ID'] = incisiondf['SEZ_ID'].astype(int)

#calculate year column 
incisiondf['Year'] = incisiondf['survey_date'].dt.year

incisiondf

# Display the updated incisiondf
print(incisiondf)

#Add SEZ_ID column using lookup dictionary
#incisiondf['SEZ_ID'] = SEZIDdf['GlobalID'].map(lambda x: lookup_dict.get(x, {}).get('SEZ_ID', None))

# Fill NaN values with a specific value, such as 0
#incisiondf['SEZ_ID'] = incisiondf['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
#incisiondf['SEZ_ID'] = incisiondf['SEZ_ID'].astype(int)

#incisiondf = incisiondf.merge(df, on='Assessment_Unit_Name', how='left')




#----------------------------------------------------------------#
#Grade, Score
#----------------------------------------------------------------#
incisiondf['Incision_Rating']=incisiondf['incision_ratio'].apply(categorize_incision)
incisiondf['Incision_Score']= incisiondf['Incision_Rating'].apply(score_indicator)

incisiondf['Incision_Data_Source'] = 'TRPA'

incisiondf.head()
#----------------------------------------------------------------#
#post ending dataframe to incision called stage_incision GDB location
#----------------------------------------------------------------#
# Define the name of the feature class
feature_class_name = 'incision'

# Define the full path to the feature class
feature_class_path = stage_incision 

#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'Year': 'Year',
    'Incision_Data_Source': 'Incision_Data_Source',
    'incision_ratio': 'Incision_Ratio',
    'Incision_Rating': 'Incision_Rating',
    'Incision_Score': 'Incision_Score',
    'SEZ_ID': 'SEZ_ID'
}

# Rename fields based on field mappings
incisionfinaldf = incisiondf.rename(columns=field_mapping).drop(columns=[col for col in incisiondf.columns if col not in field_mapping])

readyincisiondf = incisionfinaldf.groupby(['SEZ_ID', 'Year']).first().reset_index()

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(readyincisiondf)

#Write dataframe to fc in SEZ_Data.GDB
# Convert DataFrame to a list of dictionaries
data = readyincisiondf.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(readyincisiondf.columns)

# Append data to existing table
#with arcpy.da.InsertCursor(stage_incision, field_names) as cursor:
 #   for row in data:
  #      cursor.insertRow([row[field] for field in field_names])

#print("Data appended to table successfully.")

                  Assessment_Unit_Name  incision_ratio         survey_date  \
0             Angora Creek - tributary             NaN 2023-05-24 12:00:00   
1                   Angora meadows - 8             NaN 2023-05-24 12:00:00   
2             Angora Creek - tributary             NaN 2023-05-24 12:00:00   
3             Glenbrook Creek - middle             NaN 2023-05-30 12:00:00   
4                Glenbrook meadows - 1             1.0 2023-05-30 12:00:00   
..                                 ...             ...                 ...   
230              Meeks Bay meadows - 2             1.0 2023-10-04 12:00:00   
231           Angora Creek - tributary             NaN 2024-06-18 12:00:00   
232              Meeks Bay meadows - 4             NaN 2023-10-04 12:00:00   
233         Blackwood Creek - middle 4             NaN 2023-09-18 12:00:00   
234  Heavenly Valley Creek meadows - 2             NaN 2024-06-17 12:00:00   

     SEZ_ID  Year  
0       519  2023  
1        88  2023  
2  

In [25]:
# Merge readyincisiondf with SEZinfo_df to add the SEZ_Type
readyincisiondf = readyincisiondf.merge(dfSEZ[['SEZ_ID', 'SEZ_Type']], on='SEZ_ID', how='left')

### Invasive Species

In [ ]:
#Calfora Data points--dates only go to 2012?
# dates only go to 2012 hereCArest = "https://map.dfg.ca.gov/arcgis/rest/services/Project_BIOS_Public/q_BIOS_Public_pointslines07/MapServer/1"
#CArest = "https://map.dfg.ca.gov/arcgis/rest/services/Project_BIOS_Public/q_BIOS_Public_pointslines27/MapServer/0"
# Create a feature layer object
#cafeature_layer = FeatureLayer(CArest)
#Query caflora layer to that it only shows 2020-2023 or only county?
#start_year = 'January 1, 2020'
#end_year= 'December 31, 2023'
# Convert to datetime objects
#start_date = pd.to_datetime(start_year)
#end_date = pd.to_datetime(end_year)

#Query before or after??
#query = (CAsdf['Date_']>= start_date) & (CAsdf['Date_']<= end_date)

#CA_filtereddf = CAsdf.join[query].copy()

#### Code using REST Service- most likely will reuse this one

In [ ]:

import pandas as pd
from arcgis.features import FeatureLayer
from utils import get_fs_data_spatial_query

# Define the USFS REST endpoint
usfsrest = "https://apps.fs.usda.gov/arcx/rest/services/EDW/EDW_InvasiveSpecies_01/MapServer/0"
where    = "FS_UNIT_ID = '0519'"

# Query the feature layer
sdfUSFS = get_fs_data_spatial_query(usfsrest, where)

In [ ]:
#spatial join of sdf and sez master


# Create the spatially enabled DataFrame (sdf) for target feature SEZ assessment units
SEZsdf = pd.DataFrame.spatial.from_featureclass(SEZ_Master)
#spatial reference stuff
sdfUSFS.spatial.sr = SEZsdf.spatial.sr

#CAsdf.spatial.set_spatial_reference(SEZsdf.spatial.sr)
#perform spatial join
usfsdata = SEZsdf.spatial.join(sdfUSFS, how="inner")

In [ ]:
print(invasivedf['plant_type'].unique())

In [ ]:
#----------------------------------------
#SET UP DATA WRANGLE
#-----------------------------------------

#Path to external data usfs with rest service--This assumes rest service is up to date to 2023

# Paths to the feature classes
invasive19fc = os.path.join(invasiveplant19gdb, "Invasive_Species_2019")
invasive20fc = os.path.join(invasiveplant20gdb, "Invasive_Species_2020")
invasive22fc = os.path.join(invasiveplant22gdb, "Invasive_Species_2022")
invasive23fc = os.path.join(invasiveplant23gdb, "sez_invasive_plant")


#Use gdb because they have assessment unit name in them... 2022 i had to manually add in PRO
invasive23fields = ['Assessment_Unit', 'invasives_percent_cover','invasives_plant_type', 'Species_Level', 'invasive_type_other', 'created_date']
invasive22fields = ['Assessment_Unit_Name', 'Invasives_Plant_Type', 'Invasives_Percent_Cover', 'InvasiveType_Other', 'Survey_Date']
invasive20fields = ['Assessment_Unit_Name', 'Invasives_Plant_Type', 'Invasives_Percent_Cover',  'Other', 'Survey_Date']
invasive19fields = ['SITE_ID', 'SURVEY_DATE', 'INVASIVE_PLANT', 'PERCENT_COVER' ]
usfsfields = ['Assessment_Unit_Name', 'COMMON_NAME', 'SCIENTIFIC_NAME']

# Initialize an empty list to store data
data = []


# Function to read feature class into DataFrame
def feature_class_to_dataframe(feature_class, fields):
    data = [row for row in arcpy.da.SearchCursor(feature_class, fields)]
    return pd.DataFrame(data, columns=fields)

# Read feature classes into DataFrames
# Read feature classes into DataFrames
invasive19df = feature_class_to_dataframe(invasive19fc, invasive19fields)
invasive20df = feature_class_to_dataframe(invasive20fc, invasive20fields)
invasive22df = feature_class_to_dataframe(invasive22fc, invasive22fields)
invasive23df = feature_class_to_dataframe(invasive23fc, invasive23fields)
# Create a new DataFrame with only the specified columns
# Convert SpatialDataFrame to DataFrame

usfsdf = usfsdata[usfsfields]
#usfsdf = usfsdata.drop(columns='SHAPE')

print(usfsdf)
#usfs23df = feature_class_to_dataframe(usfsdata, usfsfields)


invasive19df['Source'] = 'TRPA'
invasive20df['Source'] = 'TRPA'
invasive22df['Source'] = 'TRPA'
invasive23df['Source'] = 'TRPA'
usfsdf['Source'] = 'USFS'

#usfsdf['Year'] = '2023'


# Rename fields
invasive19df.rename(columns={'SITE_ID': 'Assessment_Unit_Name', 'INVASIVE_PLANT': 'plant_type', 'PERCENT_COVER': 'percent_cover', 'SURVEY_DATE': 'created_date','Source':'Source'}, inplace=True)
invasive20df.rename(columns={'Assessment_Unit_Name': 'Assessment_Unit_Name', 'Invasives_Plant_Type': 'plant_type', 'Other': 'other', 'Invasives_Percent_Cover': 'percent_cover', 'Survey_Date': 'created_date', 'Source':'Source'}, inplace=True)
invasive22df.rename(columns={'Assessment_Unit': 'Assessment_Unit_Name', 'Invasives_Plant_Type': 'plant_type', 'InvasiveType_Other': 'other', 'Invasives_Percent_Cover': 'percent_cover', 'Survey_Date': 'created_date','Source':'Source'}, inplace=True)
invasive23df.rename(columns={'Assessment_Unit': 'Assessment_Unit_Name', 'invasives_percent_cover': 'percent_cover', 'invasives_plant_type': 'plant_type', 'invasive_type_other': 'other', 'created_date': 'created_date','Source':'Source'}, inplace=True)
usfsdf.rename(columns={'Assessment_Unit_Name': 'Assessment_Unit_Name', 'COMMON_NAME': 'plant_type', 'Source':'Source'}, inplace=True)
#Remove null plant types for usfs data
usfsdf = usfsdf[~usfsdf['plant_type'].isna()]


# Concatenate DataFrames
invasivedf = pd.concat([usfsdf, invasive19df, invasive20df, invasive22df, invasive23df], ignore_index=True)
# Display merged DataFrame
print(invasivedf)


#Define SEZ ID based on Assessment_Unit_Name for QA on SEZ Name 
invasivedf['SEZ_ID'] = invasivedf['Assessment_Unit_Name'].map(lookup_dict)

# Fill NaN values with a specific value, such as 0
invasivedf['SEZ_ID'] = invasivedf['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
#invasivedf['SEZ_ID'] = invasivedf['SEZ_ID'].astype(int)
invasivedf['Assessment_Unit_Name'] = invasivedf['Assessment_Unit_Name'].replace({'Blackwood Creek - upper 2': 'Blackwood Creek - Upper 2', 'Taylor Creek marsh - 1': 'Taylor Creek marsh', 'Sky Meadows meadow - 1': 'Sky meadows'})
# Remove rows where data source is USFS and plant type is NaN
#invasivedf = invasivedf[~((invasivedf['Source'] == 'USFS') & invasivedf['plant_type'].isna())]

# Set 'Year' column based on data source

invasivedf.loc[invasivedf['Source'] == 'USFS', 'Year'] = '2023'
invasivedf.loc[invasivedf['Source'] == 'TRPA', 'Year'] = invasivedf['created_date'].dt.year

#invasivedf.reset_index(drop=True, inplace=True)
invasivedf.reset_index(drop=True, inplace=True)

#invasivedf = invasivedf.drop_duplicates()


In [ ]:
#skip this
invasivedf_values_unique = invasivedf.values.flatten()
is_unique = len(invasivedf_values_unique) == len(set(invasivedf_values_unique))
print(is_unique)

# Check for duplicate rows
duplicate_rows = invasivedf[invasivedf.duplicated(keep=False)]

# Display duplicate rows
print(duplicate_rows)


In [ ]:
#---------------------------#
    #Prep Plant_type Data
#---------------------------#
#Make a dataframe to capture 'other' plants in trpa data and then add it to invasive df
other_plants_df = invasivedf[['Source', 'Year', 'SEZ_ID', 'Assessment_Unit_Name', 'Year', 'other']].copy()

#Get rid of Null values
other_plants_df = other_plants_df[~other_plants_df['other'].isna()]

#invasivedf.reset_index(drop=True, inplace=True)
other_plants_df.reset_index(drop=True, inplace=True)


#Rename 'other to plant_type
other_plants_df.rename(columns={'other': 'plant_type'}, inplace=True)

#invasivedf = invasivedf.drop_duplicates()


# Concatenate other_plants_df with invasivedf JUST DO THIS MANUALLY 
#invasivedf = pd.concat([invasivedf, other_plants_df], ignore_index=True)

#Append
#invasivesdf=invasivedf.append(other_plants_df)
#Concatenate the new DataFrame with the existing invasivedf
#invasivedf = pd.concat([invasivedf, other_plants_df], ignore_index=True)


# Replace various representations of null values with 'none'
null_representations = ['<null>', '<Null>', '', 'NA', 'N/A', 'nan', 'NaN', 'None', 'NULL', None]
invasivedf['plant_type'] = invasivedf['plant_type'].replace(null_representations, 'none')

# Split plant types by comma and create new rows
invasivedf['plant_type'] = invasivedf['plant_type'].str.split(pat=',')
invasivedf = invasivedf.explode('plant_type')

# Capitalize the first word and replace underscores with spaces
invasivedf['plant_type'] = invasivedf['plant_type'].str.split('_').str[0].str.capitalize() + ' ' + invasivedf['plant_type'].str.split('_').str[1:].str.join(' ')

# Remove spaces after the words and capitalize the first word while replacing underscores with spaces
invasivedf['plant_type'] = invasivedf['plant_type'].astype(str)


# Remove any spaces at the beginning or end of a word
invasivedf['plant_type'] = invasivedf['plant_type'].str.strip()


#---------------------#
#These replacements are so the same weed with different common name doesn't get counted twice toward score MAY NEED TO ADD MORE DESCREPENCIES in coming years if more pop up or find more elopquent solution
#---------------------#
# Replace 'Common mullein' with 'Wooly mullein' in the 'plant_type' column 
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Common mullein', 'Wooly mullein')
# Replace 'Nodding plumeless thistle' with 'Musk thistle' in the 'plant_type' column
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Nodding plumeless thistle', 'Musk thistle')
# Replace 'Nodding plumeless thistle' with 'Musk thistle' in the 'plant_type' column
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Field bindweed', 'Common bindweed')
# Replace 'Nodding plumeless thistle' with 'Musk thistle' in the 'plant_type' column
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Common st. johnswort', 'Klamathweed')
# Replace 'Broadleaf and leaved' with 'Perennial' in the 'plant_type' column
# Define the replacements dictionary
replacements = {'Broadleaf Pepperweed': 'Perennial pepperweed', 'Broadleaved pepperweed': 'Perennial pepperweed'}

# Replace values in the 'plant_type' column using the dictionary
invasivedf['plant_type'] = invasivedf['plant_type'].replace(replacements)

#Replace Sulphur Cinquefoil with sulfur
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Sulphur cinquefoil', 'Sulfur cinquefoil')
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Sweetclover', 'White sweetclover')
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Reed canary grass', 'Reed canarygrass')
#Replace
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Salt cedar', 'Tamarisk')
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Butter and eggs', 'Yellow toadflax')
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Canada cottonthistle', 'Canada thistle')
# Replace empty strings or other placeholders with NaN
#invasivedf['plant_type'] = invasivedf['plant_type'].replace('', np.nan)

# Drop duplicates based on 'Assessment_Unit_Name' and 'Year' in the remaining DataFrame
invasivedf = invasivedf.drop_duplicates(subset=['Assessment_Unit_Name', 'Year', 'plant_type'], keep='first')


grouped_df = invasivedf.groupby(['Assessment_Unit_Name', 'Year'])['plant_type']

# Aggregate the plant types into one column separated by commas
combined_plant_types = grouped_df.apply(lambda x: ', '.join(x)).reset_index(name='all_plant_types')





In [ ]:
for group, data in invasivedf.groupby(['Assessment_Unit_Name', 'Year']):
    print(group)
    print(data)
    print('-------------------------')

In [ ]:
#----------------------------------------------------#
#Create Look up dictionary to assign priority group for each plant and then use it
#----------------------------------------------------#

# Step 1: Read the Excel file into a DataFrame
csv_data = pd.read_csv(r"F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Invasives Priority lookup.csv") 

#Define Empty look up dataframe
Invasives_lookup = {}

key = 'Common'
values = ['Scientific', 'Priority'] 

Invasives_lookup= csv_data.set_index(key)[values].to_dict(orient='index')

# Define a custom function to map plant types to priorities
def map_priority(plant_type):
    if pd.isnull(plant_type):
        return 'None' # Return NaN for NaN values
    else:
        # Extract the priority from the dictionary, or return 'Unknown' if not found
        plant_info = Invasives_lookup.get(plant_type)
        if plant_info:
            return plant_info['Priority']
        else:
            return 'Unknown'


# Create a new column 'Priority' based on the mapping from the dictionary
invasivedf['Priority'] = invasivedf['plant_type'].map(map_priority)

print(invasivedf.columns)
#invasivedf['Priority'] = pd.to_numeric(invasivedf['Priority'], errors='coerce')

# group by assessment unit and year and summarize the priority level of plants in each unit
invasive_summary = invasivedf.groupby(['Assessment_Unit_Name', 'Year', 'Priority','Source'], dropna=False).size().reset_index(name='Count')

invasive_summary_priority = invasive_summary.pivot_table(index=['Assessment_Unit_Name', 'Year','Source'], columns='Priority', values='Count', fill_value=0)


# Reset the index to flatten the DataFrame
invasive_summary_priority.reset_index(inplace=True)

In [ ]:
#----------------------------------------------------------------#
#Grade, Score, # of Invasives
#----------------------------------------------------------------#

# Process Data
def rate_invasive(priority):
    if (priority[3] + priority[4] == 1):
        return 'B'  # Assign score D
    elif (priority[3] + priority[4] == 2) or priority[1] == 1 or priority[2] == 1:
        return 'C'  # Assign score C
    elif (priority[3] + priority[4] >= 3)or priority[1] >= 2 or priority[2] >= 2 or (priority[1] + priority[2] >= 2):  
       return 'D'  # Assign score B
    else:
        return 'A'  # Assign score A (no invasives of any priority are present)

# Apply the rating function to the summary DataFrame
invasive_summary_priority['Invasives_Rating'] = invasive_summary_priority[[1, 2, 3, 4]].apply(rate_invasive, axis=1)

#Calculate the score for the sez
invasive_summary_priority['Invasives_Score']= invasive_summary_priority['Invasives_Rating'].apply(score_indicator) 

#Calculate total number of invasives per sez per year
invasive_summary_priority['Number_of_Invasives']= invasive_summary_priority[[1, 2, 3, 4]].sum(axis=1)

#Define SEZ ID based on Assessment_Unit_Name for QA on SEZ Name 
invasive_summary_priority['SEZ_ID'] = invasive_summary_priority['Assessment_Unit_Name'].map(lookup_dict)

invasive_summary_priority['all_plants']= combined_plant_types['all_plant_types']



In [ ]:
#----------------------------------------------------------------#
#Prep and post ending dataframe to invasives table in SEZ_Data.GDB
#----------------------------------------------------------------#
#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'Year': 'Year',
    'Source': 'Invasives_Data_Source',
    'Number_of_Invasives': 'Invasives_Number_of_Invasives',
    'Invasives_Rating': 'Invasives_Rating',
    'Invasives_Score': 'Invasives_Scores',
    'SEZ_ID': 'SEZ_ID',
    'percent_cover': 'Invasives_Percent_Cover',
    'all_plants': 'Invasives_Plant_Types',
}

# Rename fields based on field mappings
readyinvasivedf = invasive_summary_priority.rename(columns=field_mapping).drop(columns=[col for col in invasive_summary_priority.columns if col not in field_mapping])

readyinvasivedf['SEZ_ID'] = readyinvasivedf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(readyinvasivedf)

# Convert DataFrame to a list of dictionaries
data = readyinvasivedf.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(readyinvasivedf.columns)

# Append data to existing table uncomment when ready
#with arcpy.da.InsertCursor(stage_invasives, field_names) as cursor:
 #   for row in data:
  #      cursor.insertRow([row[field] for field in field_names])

print("Data appended to table successfully.")

#### Invasive with gdb and usfs pre joined layer

In [ ]:
#Invasive Species Use if need to use GDB to import data--shouldn't have to


#Path to external data usfs 
usfsdata = os.path.join(master_path, "usfsinvasivesez24")
#Path to external data Calflora aka State Park Data
#calfloradata = os.path.join(master_path, "")

# Paths to the feature classes
invasive19fc = os.path.join(invasiveplant19gdb, "Invasive_Species_2019")
invasive20fc = os.path.join(invasiveplant20gdb, "Invasive_Species_2020")
invasive22fc = os.path.join(invasiveplant22gdb, "Invasive_Species_2022")
invasive23fc = os.path.join(invasiveplant23gdb, "sez_invasive_plant")


#Use gdb because they have assessment unit name in them... 2022 i had to manually add in PRO
invasive23fields = ['Assessment_Unit', 'invasives_percent_cover','invasives_plant_type', 'Species_Level', 'invasive_type_other', 'created_date']
invasive22fields = ['Assessment_Unit_Name', 'Invasives_Plant_Type', 'Invasives_Percent_Cover', 'InvasiveType_Other', 'Survey_Date']
invasive20fields = ['Assessment_Unit_Name', 'Invasives_Plant_Type', 'Invasives_Percent_Cover',  'Other', 'Survey_Date']
invasive19fields = ['SITE_ID', 'SURVEY_DATE', 'INVASIVE_PLANT', 'PERCENT_COVER' ]
usfsfields = ['Assessment_Unit_Name', 'PLANT_COMM', 'SCIENTIFIC', 'DATE_COLLE', 'Eradicated']

# Initialize an empty list to store data
data = []


# Function to read feature class into DataFrame
def feature_class_to_dataframe(feature_class, fields):
    data = [row for row in arcpy.da.SearchCursor(feature_class, fields)]
    return pd.DataFrame(data, columns=fields)

# Read feature classes into DataFrames
# Read feature classes into DataFrames
invasive19df = feature_class_to_dataframe(invasive19fc, invasive19fields)
invasive20df = feature_class_to_dataframe(invasive20fc, invasive20fields)
invasive22df = feature_class_to_dataframe(invasive22fc, invasive22fields)
invasive23df = feature_class_to_dataframe(invasive23fc, invasive23fields)
usfs23df = feature_class_to_dataframe(usfsdata, usfsfields)


invasive19df['Source'] = 'TRPA'
invasive20df['Source'] = 'TRPA'
invasive22df['Source'] = 'TRPA'
invasive23df['Source'] = 'TRPA'
usfs23df['Source'] = 'USFS'

# Rename fields
invasive19df.rename(columns={'SITE_ID': 'Assessment_Unit_Name', 'INVASIVE_PLANT': 'plant_type', 'PERCENT_COVER': 'percent_cover', 'SURVEY_DATE': 'created_date','Source':'Source'}, inplace=True)
invasive20df.rename(columns={'Assessment_Unit_Name': 'Assessment_Unit_Name', 'Invasives_Plant_Type': 'plant_type', 'Other': 'other', 'Invasives_Percent_Cover': 'percent_cover', 'Survey_Date': 'created_date', 'Source':'Source1'}, inplace=True)
invasive22df.rename(columns={'Assessment_Unit': 'Assessment_Unit_Name', 'Invasives_Plant_Type': 'plant_type', 'InvasiveType_Other': 'other', 'Invasives_Percent_Cover': 'percent_cover', 'Survey_Date': 'created_date','Source':'Source1'}, inplace=True)
invasive23df.rename(columns={'Assessment_Unit': 'Assessment_Unit_Name', 'invasives_percent_cover': 'percent_cover', 'invasives_plant_type': 'plant_type', 'invasive_type_other': 'other', 'created_date': 'created_date','Source':'Source1'}, inplace=True)
usfs23df.rename(columns={'Assessment_Unit_Name': 'Assessment_Unit_Name', 'PLANT_COMM':'plant_type', 'DATE_COLLE':'created_date', 'Source':'Source2'}, inplace=True)


# Concatenate DataFrames
invasivedf = pd.concat([usfs23df, invasive19df, invasive20df, invasive22df, invasive23df], ignore_index=True)

# Display merged DataFrame
print(invasivedf)

#Define SEZ ID based on Assessment_Unit_Name
invasivedf['SEZ_ID'] = invasivedf['Assessment_Unit_Name'].map(lookup_dict)

# Fill NaN values with a specific value, such as 0
invasivedf['SEZ_ID'] = invasivedf['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
invasivedf['SEZ_ID'] = invasivedf['SEZ_ID'].astype(int)
invasivedf['Assessment_Unit_Name'] = invasivedf['Assessment_Unit_Name'].replace({'Blackwood Creek - upper 2': 'Blackwood Creek - Upper 2', 'Taylor Creek marsh - 1': 'Taylor Creek marsh', 'Sky Meadows meadow - 1': 'Sky meadows'})

#calculate year column 
invasivedf['Year'] = invasivedf['created_date'].dt.year


#---------------------------#
    #Prep Data
#---------------------------#
# Replace 'other' or 'Other' in 'plant_type' column with values from 'other' column
invasivedf['plant_type'] = invasivedf.apply(lambda row: row['other'] if pd.notna(row['plant_type']) and row['plant_type'].lower() in ['other', 'Other'] else row['plant_type'], axis=1)

# Drop the 'other' column
invasivedf.drop(columns=['other'], inplace=True)

# Function to separate plant types and create new rows
def separate_species(df):
    # Split plant types by comma and create new rows
    df['plant_type'] = df['plant_type'].str.split(',')
    df = df.explode('plant_type')
    return df

# Apply the function to your DataFrame
invasivedf = separate_species(invasivedf)

# Remove rows where data source is USFS and plant type is NaN
invasivedf = invasivedf[~((invasivedf['Source'] == 'USFS') & invasivedf['plant_type'].isna())]

# Capitalize the first word and replace underscores with spaces
invasivedf['plant_type'] = invasivedf['plant_type'].str.split('_').str[0].str.capitalize() + ' ' + invasivedf['plant_type'].str.split('_').str[1:].str.join(' ')

# Remove spaces after the words and capitalize the first word while replacing underscores with spaces
invasivedf['plant_type'] = invasivedf['plant_type'].astype(str)


# Remove any spaces at the beginning or end of a word
invasivedf['plant_type'] = invasivedf['plant_type'].str.strip()


# Remove rows where data source is USFS and plant type is NaN
invasivedf = invasivedf[~((invasivedf['Source'] == 'USFS') & invasivedf['plant_type'].isna())]
#Remove Eradicated 

# Filter out rows where 'eradicated' column is 'Yes'
invasivedf = invasivedf[invasivedf['Eradicated'] != 'Yes']

#These replacements are so the same weed with different common name doesn't get counted twice toward score MAY NEED TO ADD MORE DESCREPENCIES in coming years if more pop up or find more elopquent solution
# Replace 'Common mullein' with 'Wooly mullein' in the 'plant_type' column 
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Common mullein', 'Wooly mullein')
# Replace 'Nodding plumeless thistle' with 'Musk thistle' in the 'plant_type' column
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Nodding plumeless thistle', 'Musk thistle')
# Replace 'Nodding plumeless thistle' with 'Musk thistle' in the 'plant_type' column
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Field bindweed', 'Common bindweed')
# Replace 'Nodding plumeless thistle' with 'Musk thistle' in the 'plant_type' column
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Common st. johnswort', 'Klamathweed')
# Replace 'Broadleaf and leaved' with 'Perennial' in the 'plant_type' column
# Define the replacements dictionary
replacements = {'Broadleaf Pepperweed': 'Perennial pepperweed', 'Broadleaved pepperweed': 'Perennial pepperweed'}

# Replace values in the 'plant_type' column using the dictionary
invasivedf['plant_type'] = invasivedf['plant_type'].replace(replacements)

#Replace Sulphur Cinquefoil with sulfur
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Sulphur cinquefoil', 'Sulfur cinquefoil')
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Sweetclover', 'White sweetclover')

#Replace
invasivedf['plant_type'] = invasivedf['plant_type'].replace('Salt cedar', 'Tamarisk')

# Now, drop duplicates based on the specified subset of columns
invasivedf = invasivedf.drop_duplicates(subset=['SEZ_ID', 'Year', 'plant_type'])

# Reset index if needed
invasivedf.reset_index(drop=True, inplace=True)


# Remove duplicates based on SEZ, Year, and plant_type
invasivedf = invasivedf.drop_duplicates(subset=['SEZ_ID', 'Year', 'plant_type'])
#----------------------------------------------------#
#Create Look up dictionary to assign priority group for each plant and then use it
#----------------------------------------------------#

# Step 1: Read the Excel file into a DataFrame
csv_data = pd.read_csv(r"F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Invasives Priority lookup.csv") 

#Define Empty look up dataframe
Invasives_lookup = {}

key = 'Common'
values = ['Scientific', 'Priority'] 

Invasives_lookup= csv_data.set_index(key)[values].to_dict(orient='index')

# Define a custom function to map plant types to priorities
def map_priority(plant_type):
    if pd.isnull(plant_type):
        return np.nan  # Return NaN for NaN values
    else:
        # Extract the priority from the dictionary, or return 'Unknown' if not found
        plant_info = Invasives_lookup.get(plant_type)
        if plant_info:
            return plant_info['Priority']
        else:
            return 'Unknown'

#Create a new column [Scientific based on look up dictionary
#invasivedf['Scientific']=invasivedf['plant_type'].map(map_priority)

# Create a new column 'Priority' based on the mapping from the dictionary
invasivedf['Priority'] = invasivedf['plant_type'].map(map_priority)

#invasivedf['Priority'] = pd.to_numeric(invasivedf['Priority'], errors='coerce')
# group by assessment unit and year and summarize the priority level of plants in each unit
invasive_summary = invasivedf.groupby(['Assessment_Unit_Name', 'Year', 'Priority']).size().reset_index(name='Count')

invasive_summary_priority2 = invasive_summary.pivot_table(index=['Assessment_Unit_Name', 'Year'], columns='Priority', values='Count', fill_value=0)

# Reset the index to flatten the DataFrame
invasive_summary_priority2.reset_index(inplace=True)

#invasive_summary_priority['Source'] = invasivedf['Source']
#----------------------------------------------------------------#
#Grade, Score, # of Headcuts
#----------------------------------------------------------------#

# Process Data
def rate_invasive(priority):
    # Check if the SEZ has at least one large headcut
    if (priority[3] + priority[4] == 1):
        return 'B'  # Assign score D
    elif (priority[3] + priority[4] == 2) or priority[1] == 1 or priority[2] == 1:
        return 'C'  # Assign score C
    elif (priority[3] + priority[4] >= 3)or priority[1] >= 2 or priority[2] >= 2 or (priority[1] + priority[2] >= 2):  
       return 'B'  # Assign score B
    else:
        return 'A'  # Assign score A (no invasives of any priority are present)

# Apply the rating function to the summary DataFrame
invasive_summary_priority2['Invasives_Rating'] = invasive_summary_priority2.apply(rate_invasive, axis=1)

#Calculate total number of invasives per sez per year
invasive_summary_priority2['Number_of_Invasives']= invasive_summary_priority2[[1, 2, 3, 4,'Unknown']].sum(axis=1)

#Calculate the score for the sez
invasive_summary_priority2['Invasives_Score']= invasive_summary_priority2['Invasives_Rating'].apply(score_indicator)    

# make a columns in invasive summary that totals up percent cover per sez/year
invasive_summary_priority2['Invasives_Percent_Cover'] = invasivedf.groupby(['SEZ_ID', 'Year'])['percent_cover'].sum().reset_index(drop=True)

# combine the source column so that it shows all data sources that contributed to the data
# Group invasivedf by 'SEZ_ID' and 'Year' and concatenate 'Data Source 1' values
#data_source_1_combined = invasivedf.groupby(['SEZ_ID', 'Year'])['Source 1'].apply(lambda x: ', '.join(x.dropna())).reset_index()

# Group invasivedf by 'SEZ_ID' and 'Year' and concatenate 'Data Source 2' values
#data_source_2_combined = invasivedf.groupby(['SEZ_ID', 'Year'])['Source 2'].apply(lambda x: ', '.join(x.dropna())).reset_index()

# Merge data_source_1_combined and data_source_2_combined on 'SEZ_ID' and 'Year'
#merged_data_sources = pd.merge(data_source_1_combined, data_source_2_combined, on=['SEZ_ID', 'Year'], how='outer')

# Combine 'Data Source 1' and 'Data Source 2' values with a comma separator
#merged_data_sources['Data_Sources'] = merged_data_sources.apply(lambda row: ', '.join(filter(None, [row['Source 1'], row['Source 2']])), axis=1)

# Drop the individual 'Data Source 1' and 'Data Source 2' columns
#merged_data_sources.drop(columns=['Source 1', 'Source 2'], inplace=True)

# Merge merged_data_sources with invasive_summary_priority on 'SEZ_ID' and 'Year'
#invasive_summary_priority = pd.merge(invasive_summary_priority, merged_data_sources, on=['SEZ_ID', 'Year'], how='left')

#invasive_summary_priority['Data_Sources']= invasivedfinvasivedf.groupby(['SED_ID', 'Year'])[Data Source 1 ] merge with DataSource 1 separate with comma if there are both 
#----------------------------------------------------------------#
#post ending dataframe to incision called stage_incision GDB location
#----------------------------------------------------------------#


#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'Year': 'Year',
    'Data_Sources': 'Invasives_Data_Source',
    'Number_of_Invasives': 'Invasives_Number_of_Invasives',
    'Invasives_Rating': 'Invasives_Rating',
    'Invasives_Score': 'Invasives_Scores',
    'SEZ_ID': 'SEZ_ID',
    'percent_cover': 'Invasives_Percent_Cover',
    'plant_type': 'Invasives_Plant_Type',
}

# Rename fields based on field mappings
#readydf = invasive_summary_priority.rename(columns=field_mapping).drop(columns=[col for col in headcut_summary_sml.columns if col not in field_mapping])

#readydf['SEZ_ID'] = readydf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

#print(readydf)

# Convert DataFrame to a list of dictionaries
#data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
#field_names = list(readydf.columns)

# Append data to existing table
#with arcpy.da.InsertCursor(stage_headcuts, field_names) as cursor:
 #   for row in data:
  #      cursor.insertRow([row[field] for field in field_names])

#print("Data appended to table successfully.")


In [ ]:
# Check the data type of 'plant_type' column in the DataFrame
print("Data type of 'plant_type' column in DataFrame:", invasivedf['plant_type'].dtype)

# Check the data type of values in the lookup dictionary
for key, value in Invasives_lookup.items():
    print("Data type of value for key", key, "in lookup dictionary:", type(value))


### Headcuts


In [23]:
#Headcuts
#--------------------------------#
#Get Data from GDB's with assessment unit name 
#--------------------------------#

# Paths to the feature classes
headcut19fc = os.path.join(headcut19gdb, "Stream_Headcut_2019")
headcut20fc = os.path.join(headcut20gdb, "Stream_Headcut_2020")
headcut22fc = os.path.join(headcut22gdb, "Stream_Headcut_2022")
headcut23fc = os.path.join(headcut23gdb, "sez_stream_headcut")


#Use gdb because they have assessment unit name in them... 2022 i had to manually add in PRO
headcut23fields = ['Assessment_Unit', 'headcut_depth', 'created_date']
headcut22fields = ['Assessment_Unit', 'Headcut_Depth', 'synced_date']
headcut20fields = ['Assessment_Unit_Name', 'Headcut_Depth','Survey_Date']
headcut19fields = ['SITE_NAME', 'HEADCUT_DEPTH', 'SURVEY_DATE' ]
# Initialize an empty list to store data
data = []


# Function to read feature class into DataFrame
def feature_class_to_dataframe(feature_class, fields):
    data = [row for row in arcpy.da.SearchCursor(feature_class, fields)]
    return pd.DataFrame(data, columns=fields)

# Read feature classes into DataFrames
# Read feature classes into DataFrames
headcut19df = feature_class_to_dataframe(headcut19fc, headcut19fields)
headcut20df = feature_class_to_dataframe(headcut20fc, headcut20fields)
headcut22df = feature_class_to_dataframe(headcut22fc, headcut22fields)
headcut23df = feature_class_to_dataframe(headcut23fc, headcut23fields)

# Rename fields
headcut19df.rename(columns={'SITE_NAME': 'Assessment_Unit_Name', 'HEADCUT_DEPTH': 'headcut_depth', 'SURVEY_DATE': 'created_date'}, inplace=True)
headcut20df.rename(columns={'Assessment_Unit_Name': 'Assessment_Unit_Name', 'Headcut_Depth': 'headcut_depth', 'Survey_Date': 'created_date'}, inplace=True)
headcut22df.rename(columns={'Assessment_Unit': 'Assessment_Unit_Name', 'Headcut_Depth': 'headcut_depth', 'synced_date': 'created_date'}, inplace=True)
headcut23df.rename(columns={'Assessment_Unit': 'Assessment_Unit_Name', 'headcut_depth': 'headcut_depth', 'created_date': 'created_date'}, inplace=True)

# Concatenate DataFrames
headcutdf = pd.concat([headcut19df, headcut20df, headcut22df, headcut23df], ignore_index=True)

# Display merged DataFrame
print(headcutdf)

#hopefully this is just one time


#--------------------------------#
#Get Data from exported table where Assessment Unit is joined---parentglobalids don't match up for years before 2023. need to redo sde.collect 2019-2022 data append.... 
#--------------------------------#

#--------------------------------#
#Get Data from SDE.Collect...instead of GDB
#--------------------------------#
#----------------------------------------------------------------#
#Add correct info to dataframe
#----------------------------------------------------------------#
# Create a new column 'SEZ ID'
#This code is for the excel look up dictionary---ADD last? this is goo dfor QA on assessment unit names because 0's tell you the name is wrong
headcutdf['SEZ_ID'] = headcutdf['Assessment_Unit_Name'].map(lookup_dict)


# Fill NaN values with a specific value, such as 0
headcutdf['SEZ_ID'] = headcutdf['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
headcutdf['SEZ_ID'] = headcutdf['SEZ_ID'].astype(int)
headcutdf['Assessment_Unit_Name'] = headcutdf['Assessment_Unit_Name'].replace({'Blackwood Creek - upper 2': 'Blackwood Creek - Upper 2', 'Taylor Creek marsh - 1': 'Taylor Creek marsh', 'Sky meadows - 1': 'Sky Meadows'})

#calculate year column 
headcutdf['Year'] = headcutdf['created_date'].dt.year

#----------------------------------------------#
# Process Data
#----------------------------------------------#
# assign small, medium, large to headcut
headcutdf['Headcut_Size']=headcutdf['headcut_depth'].apply(categorize_headcut)


# Group by 'SEZ_ID', 'Year', and 'Headcut_Size', and count the number of occurrences for each group
headcut_summary = headcutdf.groupby(['Assessment_Unit_Name', 'Year', 'Headcut_Size']).size().reset_index(name='Count')

print(type(headcut_summary))
# Pivot the table to have 'Headcut_Size' categories as columns
headcut_summary_sml = headcut_summary.pivot_table(index=['Assessment_Unit_Name', 'Year'], columns='Headcut_Size', values='Count', fill_value=0)

# Reset the index to flatten the DataFrame
headcut_summary_sml.reset_index(inplace=True)





# Display the summarized DataFrame
print(headcut_summary_sml)

#----------------------------------------------------------------#
#Grade, Score, # of Headcuts
#----------------------------------------------------------------#
    
    # Process Data
def rate_headcut(row):
    # Check if the SEZ has at least one large headcut
    if row['large'] >= 1:
        return 'D'  # Assign score D
    elif row['medium'] >= 1:
        return 'C'  # Assign score C
    elif row['small'] >= 1:
       return 'B'  # Assign score B
    else:
        return 'A'  # Assign score A (no headcuts)

# Apply the rating function to the summary DataFrame
headcut_summary_sml['Headcuts_Rating'] = headcut_summary_sml.apply(rate_headcut, axis=1)

#Calculate total number of headcuts per sez per year
headcut_summary_sml['Number_of_Headcuts']= headcut_summary_sml[['large', 'medium', 'small']].sum(axis=1)

#Calculate the score for the sez
headcut_summary_sml['Headcuts_Score']= headcut_summary_sml['Headcuts_Rating'].apply(score_indicator)

#Add Datasource
headcut_summary_sml['Headcuts_Data_Source'] = 'TRPA'


#----------------------------------------------------------------#
#post ending dataframe to incision called stage_incision GDB location
#----------------------------------------------------------------#


#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'Year': 'Year',
    'Headcuts_Data_Source': 'Headcuts_Data_Source',
    'Number_of_Headcuts': 'Number_of_Headcuts',
    'Headcuts_Rating': 'Headcuts_Rating',
    'Headcuts_Score': 'Headcuts_Score',
    'SEZ_ID': 'SEZ_ID',
    'small': 'small',
    'medium': 'medium',
    'large': 'large'
}

# Rename fields based on field mappings
readydf = headcut_summary_sml.rename(columns=field_mapping).drop(columns=[col for col in headcut_summary_sml.columns if col not in field_mapping])

readydf['SEZ_ID'] = readydf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(readydf)

# Convert DataFrame to a list of dictionaries
data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(readydf.columns)

# Append data to existing table
with arcpy.da.InsertCursor(stage_headcuts, field_names) as cursor:
    for row in data:
        cursor.insertRow([row[field] for field in field_names])

print("Data appended to table successfully.")



                      Assessment_Unit_Name  headcut_depth        created_date
0                   Susquehana meadows - 1           0.50 2019-05-13 00:00:00
1                   Susquehana meadows - 1           0.50 2019-05-13 00:00:00
2                  Golden Bear meadows - 1           0.60 2019-05-13 00:00:00
3                  Golden Bear meadows - 1           0.40 2019-05-13 00:00:00
4                  Golden Bear meadows - 1           0.70 2019-05-13 00:00:00
..                                     ...            ...                 ...
601     Trout Creek headwaters meadows - 1           0.40 2023-08-18 18:10:40
602     Trout Creek headwaters meadows - 1           0.44 2023-08-18 18:10:40
603  Trout Creek meadows - above Pioneer 4           1.30 2023-08-30 23:05:09
604               Incline Lake meadows - 2           0.50 2023-09-18 15:26:54
605               Incline Lake meadows - 2           1.30 2023-09-18 15:26:54

[606 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>
He

In [ ]:
# Bring in External Data from USFS and Calflora

In [ ]:
#--------------------------------#
#Get Data from external sources and spatially join to our data
#--------------------------------#


#Calfora Data
CArest = "https://map.dfg.ca.gov/arcgis/rest/services/Project_BIOS_Public/q_BIOS_Public_pointslines07/MapServer/1"
#CAfc = os.path.join(CArest, "Non-Native and Invasive Plants - Calflora")
CAsdf = pd.DataFrame.spatial.from_layer(CArest)
# Delete the existing feature layer if it exists

CA_fc = arcpy.MakeFeatureLayer_management(CArest, "Calfora_invasives")


usfsrest= "https://apps.fs.usda.gov/arcx/rest/services/EDW/EDW_InvasiveSpecies_01/MapServer/0"
#usfsfc =os.path.join(usfsrest, "Current Invasive Plant Locations") 
#usfssdf = pd.DataFrame.spatial.from_layer(usfsrest)
if arcpy.Exists("usfs_layer"):
    arcpy.Delete_management("usfs_layer")
usfs_fc = arcpy.MakeFeatureLayer_management(usfsrest, "usfs_layer")
#Spatially join usfs layers to see if any of the points or polygons intersect with our sez's123

# Set the workspace environment
arcpy.env.workspace = r"F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Data Management 2023.gdb"

# Define the target and join layers
target_feature = SEZ_Master #"AssessmentUnit_Master"
CAjoin_feature = CA_fc
USFSjoin_feature = usfs_fc

# Define the output feature class
out_feature_class = r"F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Data Management 2023.gdb\externalinvasivesez"

# Define the fields you want to keep from both layers
#field_mappings = arcpy.FieldMappings()
#field_mappings.addTable(target_features)
#field_mappings.addTable(join_features)

# Keep only specific fields from the target layer
#keep_fields_target = ["Assessment_Unit_Name"]
#for field in field_mappings.fields:
 #   if field.name not in keep_fields_target:
  #      field_mappings.removeFieldMap(field_mappings.findFieldMapIndex(field.name))

#Keep only specific fields from the calfora join layer
#keep_fields_CA = ["CName", "SName", "Source", "Obs_Date"]
#for field in field_mappings.fields:
 #   if field.name not in keep_fields_CA:
  #      field_mappings.removeFieldMap(field_mappings.findFieldMapIndex(field.name))

# Keep only specific fields from the usfs join_features
#keep_fields_USFS = ["SCIENTIFIC_NAME", "COMMON_NAME", "DATE_COLLECTED"]
#for field in field_mappings.fields:
 #   if field.name not in keep_fields_USFS:
  #      field_mappings.removeFieldMap(field_mappings.findFieldMapIndex(field.name))


# Perform spatial join with specified field mappings
arcpy.analysis.SpatialJoin(
    target_features=SEZ_Master,
    join_features=[CAjoin_feature],
    out_feature_class=out_feature_class,
    join_operation="JOIN_ONE_TO_MANY",
    join_type="KEEP_ALL"
    #field_mapping=field_mappings
)

#Create Dataframe for spatially joined data
# Define the fields you want to retrieve from the output feature class
fields = ["Assessment_Unit_Name", "CName", "SName", "Source", "Obs_Date", "SCIENTIFIC_NAME", "COMMON_NAME", "DATE_COLLECTED"]

# Create a DataFrame to store the data
data = []

# Use SearchCursor to read data from the output feature class
with arcpy.da.SearchCursor(out_feature_class, fields) as cursor:
    for row in cursor:
        data.append(row)

# Convert the list of tuples into a pandas DataFrame
df = pd.DataFrame(data, columns=fields)

# Display the DataFrame
print(df)


### Bioassessment/ Biotic Integrity

In [ ]:
#Bioassessment scores- get all stream data into sde.Stream first... then look at Biotic Integrit Data Source to find which stream was used to evaluate each meadow.. this will help with percent of stream miles I believe

#----------------------------------------------------------------#
#Get Data-- all the data we need is in sde.collect.sde.survey.sde.sez_survey
#----------------------------------------------------------------##Convert feature class in gdb to Pandas DataFrame this is for REST Service

#Create SEDF setup
#streamdata is the path to the feature class in sde
# Set the workspace to your SDE connection file
arcpy.env.workspace = streamdata
feature_class= "Stream"

# Convert feature class to a pandas DataFrame
fields = [field.name for field in arcpy.ListFields(feature_class)]

# Create DataFrame
streamsdf = pd.DataFrame.spatial.from_featureclass(feature_class, columns=fields)

# Create the spatially enabled DataFrame (sdf) for target feature SEZ assessment units
SEZsdf = pd.DataFrame.spatial.from_featureclass(SEZ_Master)
#spatial reference stuff
streamsdf.spatial.sr = SEZsdf.spatial.sr

#perform spatial join of sde.stream and sez units
thesdf = SEZsdf.spatial.join(streamsdf, how='inner')

#Notes to self, Stream Miles?
#Keep only Riverine?, this may be what the smaller polygons are for 
# Filter for SEZ type 'Riverine'
#riverine_df = bioticsdf[bioticsdf['Feature_Type'] == 'Riverine']

#if the layer contains Riverine? or just for any spatial join.. see what it does
#spatial join this layer to asessment unit master layer
#ASsessment unit master layer is called SEZ_Master



In [ ]:
#----------------------#
#DATA PREP
#----------------------#
# List of columns to keep
columns_to_keep = ['Assessment_Unit_Name', 'SEZ_Type', 'Feature_Type', 'SEZ_ID','SITE_NAME', 'COUNT_VALUE', 'YEAR_OF_COUNT', 'STATION_TYPE', 'LONGITUDE', 'LATITUDE', ]
##Try this instead
# Select only the desired columns
bioticdf = thesdf.loc[:, columns_to_keep].copy()  

#DATA PREP
# Filter for years 2020 to 2023
filtered_df = bioticdf.loc[(bioticdf['YEAR_OF_COUNT'] >= 2020) & (bioticdf['YEAR_OF_COUNT'] <= 2023)].copy()

# Define SEZ ID based on Assessment_Unit_Name for QA on SEZ Name THIS METHOD USES LARGER POLYGONES 
filtered_df.loc[:, 'SEZ_ID'] = filtered_df['Assessment_Unit_Name'].map(lookup_dict)

# Fill NaN values with a specific value, such as 0
filtered_df.loc[:, 'SEZ_ID'] = filtered_df['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
filtered_df.loc[:, 'SEZ_ID'] = filtered_df['SEZ_ID'].astype(int)

# Replace values in the 'Assessment_Unit_Name' column
filtered_df.loc[:, 'Assessment_Unit_Name'] = filtered_df['Assessment_Unit_Name'].replace({'Blackwood Creek - upper 2': 'Blackwood Creek - Upper 2', 'Taylor Creek marsh - 1': 'Taylor Creek marsh', 'Sky Meadows meadow - 1': 'Sky meadows'})

# Add data source information
filtered_df['Source'] = 'TRPA, ' + filtered_df['SITE_NAME'].astype(str) + ', ' + filtered_df['YEAR_OF_COUNT'].astype(str)

#Group by Year and Assessment Unit and Site NAME and remove duplicates

filtered_df['SITE_NAME'] = filtered_df['SITE_NAME'].str.strip()
filtered_df['YEAR_OF_COUNT'] = filtered_df['YEAR_OF_COUNT'].astype(str).str.strip().astype(int)
filtered_df['YEAR_OF_COUNT'] = pd.to_numeric(filtered_df['YEAR_OF_COUNT'], errors='coerce')


# Group by Assessment_Unit_Name, SITE_NAME, and YEAR_OF_COUNT and drop duplicates
BIdf = filtered_df.groupby(['SEZ_ID', 'SITE_NAME', 'YEAR_OF_COUNT', 'COUNT_VALUE']).apply(lambda x: x.drop_duplicates()).reset_index(drop=True)



In [ ]:
print(filtered_df['YEAR_OF_COUNT'].isna().sum())


In [ ]:
#----------------------#
#Grade and Score biotic integrity
#----------------------#

#Rate the score
#ef categorize_csci(biotic_integrity):
# Apply the rating function to the summary DataFrame
BIdf['Biotic_Rating'] = BIdf['COUNT_VALUE'].apply(categorize_csci)

#Calculate the score for the sez
BIdf['Biotic_Score']= BIdf['Biotic_Rating'].apply(score_indicator) 


In [ ]:
#----------------------------------------------------------------#
#Prep and post ending dataframe to invasives table in SEZ_Data.GDB
#----------------------------------------------------------------#
#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'YEAR_OF_COUNT': 'Year',
    'Source': 'Biotic_Integrity_Data_Source',
    'COUNT_VALUE': 'Biotic_Integrity_CSCI',
    'Biotic_Rating': 'Biotic_Integrity_Rating',
    'Biotic_Score': 'Biotic_Integrity_Score',
    'SEZ_ID': 'SEZ_ID',
}

# Rename fields based on field mappings
readydf = BIdf.rename(columns=field_mapping).drop(columns=[col for col in BIdf.columns if col not in field_mapping])

readydf['SEZ_ID'] = readydf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(readydf)

# Convert DataFrame to a list of dictionaries
data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(readydf.columns)

# Append data to existing table uncomment when ready
with arcpy.da.InsertCursor(stage_biotic_integrity, field_names) as cursor:
    for row in data:
        cursor.insertRow([row[field] for field in field_names])

print("Data appended to table successfully.")

#Delete duplicates yourself.. not that much data to go through, can't figure out why it won't remove duplicates

### Conifer Encroachment Staging Table

In [ ]:
import os
import pandas as pd
import arcpy

# Path to the geodatabase
master_path = r"F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data\SEZ_Data.gdb"

# Table name in the geodatabase
table_name = "AssessmentUnits_Master"

# Function to convert geodatabase table to DataFrame
def table_to_dataframe(table):
    fields = [field.name for field in arcpy.ListFields(table)]
    data = [row for row in arcpy.da.SearchCursor(table, fields)]
    return pd.DataFrame(data, columns=fields)

try:
    # Load the data from the geodatabase table into a DataFrame
    SEZ_Master = table_to_dataframe(os.path.join(master_path, table_name))

    # Remove leading and trailing whitespace from column names
    SEZ_Master.columns = SEZ_Master.columns.str.strip()

    # Columns to select from the table
    selected_columns = ['Conifer_Encroachment_Data_Sourc',
                        'Conifer_Encroachment_Rating',                    
                        'Conifer_Encroachment_Percent_En',
                        'Conifer_Encroachment_Score',
                        'SEZ_ID',
                        'Assessment_Unit_Name',
                        'ConiferEncroachment_Comments']

    # Create a new DataFrame with selected columns
    conifer_df = SEZ_Master[selected_columns].copy()

except KeyError as e:
    print("One or more specified columns not found in the DataFrame:", e)

except Exception as e:
    print("An error occurred:", e)



In [ ]:
#Prep Data
conifer_df['Year'] = 2020

#Field Mapping
field_mapping = {
                'Assessment_Unit_Name': 'Assessment_Unit_Name',
                'Year': 'Year',
                'Conifer_Encroachment_Data_Sourc': 'Conifer_Encroachment_Data_Source',
                'Conifer_Encroachment_Rating': 'Conifer_Encroachment_Rating',                    
                'Conifer_Encroachment_Percent_En': 'Conifer_Percent_Encroached',
                'Conifer_Encroachment_Score': 'Conifer_Encroachment_Score',
                'SEZ_ID': 'SEZ_ID',
                'ConiferEncroachment_Comments': 'ConiferEncroachment_Comments'
}

# Rename fields based on field mappings
readydf = conifer_df.rename(columns=field_mapping).drop(columns=[col for col in conifer_df.columns if col not in field_mapping])

#readydf['SEZ_ID'] = readydf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(readydf)

# Convert DataFrame to a list of dictionaries
data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(readydf.columns)

# Append data to existing table uncomment when ready
with arcpy.da.InsertCursor(stage_conifer, field_names) as cursor:
    for row in data:
        cursor.insertRow([row[field] for field in field_names])

print("Data appended to table successfully.")

### Aquatic Organism Passage STagin table

In [ ]:
print(SEZ_Master.columns)

In [ ]:
#Aquatic Organism /Fish Passage- only old data for now
import os
import pandas as pd
import arcpy

# Path to the geodatabase
master_path = r"F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data\SEZ_Data.gdb"

# Table name in the geodatabase
table_name = "AssessmentUnits_Master"

# Function to convert geodatabase table to DataFrame
def table_to_dataframe(table):
    fields = [field.name for field in arcpy.ListFields(table)]
    data = [row for row in arcpy.da.SearchCursor(table, fields)]
    return pd.DataFrame(data, columns=fields)

try:
    # Load the data from the geodatabase table into a DataFrame
    SEZ_Master = table_to_dataframe(os.path.join(master_path, table_name))

    # Remove leading and trailing whitespace from column names
    SEZ_Master.columns = SEZ_Master.columns.str.strip()

    # Columns to select from the table
    selected_columns = ['AquaticOrganismPassage_Barriers',
                        'AquaticOrganismPassage_DataSour',                    
                        'AquaticOrganismPassage_NumberOf',
                        'AquaticOrganismPassage_Rating',
                        'SEZ_ID',
                        'Assessment_Unit_Name',
                        'AquaticOrganismPassage_Score',
                        'AquaticOrganismPassage_StreamMi']

    # Create a new DataFrame with selected columns
    AOP_df = SEZ_Master[selected_columns].copy()

except KeyError as e:
    print("One or more specified columns not found in the DataFrame:", e)

except Exception as e:
    print("An error occurred:", e)



In [ ]:
#Prep Data
AOP_df['Year'] = 2020

#Field Mapping
field_mapping = {
                'Assessment_Unit_Name': 'Assessment_Unit_Name',
                'Year': 'Year',
                'AquaticOrganismPassage_Barriers': 'AOP_BarriersPerMile',
                'AquaticOrganismPassage_DataSour': 'AOP_DataSource',                    
                'AquaticOrganismPassage_NumberOf': 'AOP_NumberofBarriers',
                'AquaticOrganismPassage_Score': 'AOP_Score',
                'SEZ_ID': 'SEZ_ID',
                'AquaticOrganismPassage_StreamMi': 'AOP_StreamMiles',
                'AquaticOrganismPassage_Rating': 'AOP_Rating'
}

# Rename fields based on field mappings
readydf = AOP_df.rename(columns=field_mapping).drop(columns=[col for col in AOP_df.columns if col not in field_mapping])

#readydf['SEZ_ID'] = readydf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(readydf)

# Convert DataFrame to a list of dictionaries
data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(readydf.columns)

# Append data to existing table uncomment when ready
with arcpy.da.InsertCursor(stage_aquatic, field_names) as cursor:
    for row in data:
        cursor.insertRow([row[field] for field in field_names])

print("Data appended to table successfully.")

### Habitat Fragmentation Staging Table

In [ ]:
#Create STaging Tables for External Data Put in STagin Table

# Path to the geodatabase
master_path = r"F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data\SEZ_Data.gdb"

# Table name in the geodatabase
table_name = "AssessmentUnits_Master"

# Function to convert geodatabase table to DataFrame
def table_to_dataframe(table):
    fields = [field.name for field in arcpy.ListFields(table)]
    data = [row for row in arcpy.da.SearchCursor(table, fields)]
    return pd.DataFrame(data, columns=fields)

try:
    # Load the data from the geodatabase table into a DataFrame
    SEZ_Master = table_to_dataframe(os.path.join(master_path, table_name))

    # Remove leading and trailing whitespace from column names
    SEZ_Master.columns = SEZ_Master.columns.str.strip()

    # Columns to select from the table
    selected_columns = ['Habitat_Fragmentation_Data_Sour',
                        'Habitat_Fragmentation_Imperviou',
                        'Habitat_Fragmentation_Percent_I',
                        'Habitat_Fragmentation_Rating',
                        'Habitat_Fragmentation_Score',
                        'SEZ_ID', 
                        'Assessment_Unit_Name']

    # Create a new DataFrame with selected columns
    HabFrag_df = SEZ_Master[selected_columns].copy()

except KeyError as e:
    print("One or more specified columns not found in the DataFrame:", e)

except Exception as e:
    print("An error occurred:", e)

In [ ]:
#Prep Data
HabFrag_df['Year'] = 2020

#Field Mapping
field_mapping = {
                'Assessment_Unit_Name': 'Assessment_Unit_Name',
                'Year': 'Year',
                'Habitat_Fragmentation_Data_Sour': 'Habitat_Frag_Data_Source',
                'Habitat_Fragmentation_Percent_I': 'HAbitat_Frag_Percent_Impervious',                    
                'Habitat_Fragmentation_Score': 'Habitat_Frag_Score',
                'Habitat_Fragmentation_Imperviou': 'Habitat_Frag_Impervious_Acres',
                'SEZ_ID': 'SEZ_ID',
                'Habitat_Fragmentation_Rating': 'Habitat_Frag_Rating'
}

# Rename fields based on field mappings
readydf = HabFrag_df.rename(columns=field_mapping).drop(columns=[col for col in HabFrag_df.columns if col not in field_mapping])

#readydf['SEZ_ID'] = readydf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(readydf)

# Convert DataFrame to a list of dictionaries
data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(readydf.columns)

# Append data to existing table uncomment when ready
with arcpy.da.InsertCursor(stage_habitat, field_names) as cursor:
    for row in data:
        cursor.insertRow([row[field] for field in field_names])

print("Data appended to table successfully.")

### Ditches Staging Table

In [ ]:
#Create STaging Tables for External Data Put in STagin Table

# Path to the geodatabase
master_path = r"F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data\SEZ_Data.gdb"

# Table name in the geodatabase
table_name = "AssessmentUnits_Master"

# Function to convert geodatabase table to DataFrame
def table_to_dataframe(table):
    fields = [field.name for field in arcpy.ListFields(table)]
    data = [row for row in arcpy.da.SearchCursor(table, fields)]
    return pd.DataFrame(data, columns=fields)

try:
    # Load the data from the geodatabase table into a DataFrame
    SEZ_Master = table_to_dataframe(os.path.join(master_path, table_name))

    # Remove leading and trailing whitespace from column names
    SEZ_Master.columns = SEZ_Master.columns.str.strip()

    # Columns to select from the table
    selected_columns = ['Ditches_Data_Source',
                        'Ditches_Length',
                        'Ditches_Meadow_Length',
                        'Ditches_Percent',
                        'Ditches_Rating',
                        'Ditches_Score',
                        'SEZ_ID', 
                        'Assessment_Unit_Name']

    # Create a new DataFrame with selected columns
    Ditch_df = SEZ_Master[selected_columns].copy()

except KeyError as e:
    print("One or more specified columns not found in the DataFrame:", e)

except Exception as e:
    print("An error occurred:", e)

In [ ]:
#Prep Data
Ditch_df['Year'] = 2020

#Field Mapping
field_mapping = {
                'Assessment_Unit_Name': 'Assessment_Unit_Name',
                'Year': 'Year',
                'Ditches_Data_Source': 'Ditches_Data_Source',
                'Ditches_Length': 'Ditches_Length',                    
                'Ditches_Meadow_Length': 'Ditches_Meadow_Length',
                'Ditches_Percent': 'Ditches_Percent',
                'SEZ_ID': 'SEZ_ID',
                'Ditches_Rating': 'Ditches_Rating',
                'Ditches_Score': 'Ditches_Score'
}

# Rename fields based on field mappings
readydf = Ditch_df.rename(columns=field_mapping).drop(columns=[col for col in Ditch_df.columns if col not in field_mapping])

#readydf['SEZ_ID'] = readydf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(readydf)

# Convert DataFrame to a list of dictionaries
data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(readydf.columns)

# Append data to existing table uncomment when ready
with arcpy.da.InsertCursor(stage_ditches, field_names) as cursor:
    for row in data:
        cursor.insertRow([row[field] for field in field_names])

print("Data appended to table successfully.")

### Vegetation Vigor- old data import

In [ ]:
#Create STaging Tables for External Data Put in STagin Table

# Path to the geodatabase
master_path = r"F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data\SEZ_Data.gdb"

# Table name in the geodatabase
table_name = "AssessmentUnits_Master"

# Function to convert geodatabase table to DataFrame
def table_to_dataframe(table):
    fields = [field.name for field in arcpy.ListFields(table)]
    data = [row for row in arcpy.da.SearchCursor(table, fields)]
    return pd.DataFrame(data, columns=fields)

try:
    # Load the data from the geodatabase table into a DataFrame
    SEZ_Master = table_to_dataframe(os.path.join(master_path, table_name))

    # Remove leading and trailing whitespace from column names
    SEZ_Master.columns = SEZ_Master.columns.str.strip()

    # Columns to select from the table
    selected_columns = ['VegetationVigor_DataSource',
                        'NDVI_ID',
                        'VegetationVigor_Raw',
                        'VegetationVigor_Rating',
                        'VegetationVigor_Score',
                        'SEZ_ID', 
                        'Assessment_Unit_Name']

    # Create a new DataFrame with selected columns
    vegetation_df = SEZ_Master[selected_columns].copy()

except KeyError as e:
    print("One or more specified columns not found in the DataFrame:", e)

except Exception as e:
    print("An error occurred:", e)

In [ ]:
#Prep Data
vegetation_df['Year'] = 2020

#Field Mapping
field_mapping = {
                'Assessment_Unit_Name': 'Assessment_Unit_Name',
                'Year': 'Year',
                'VegetationVigor_DataSource': 'VegetationVigor_DataSource',
                'NDVI_ID': 'NDVI_ID',                    
                'VegetationVigor_Raw': 'VegetationVigor_Raw',
                'VegetationVigor_Rating': 'VegetationVigor_Rating',
                'SEZ_ID': 'SEZ_ID',
                'VegetationVigor_Score': 'VegetationVigor_Score'
}

# Rename fields based on field mappings
readydf = vegetation_df.rename(columns=field_mapping).drop(columns=[col for col in vegetation_df.columns if col not in field_mapping])

#readydf['SEZ_ID'] = readydf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(readydf)

# Convert DataFrame to a list of dictionaries
data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(readydf.columns)

# Append data to existing table uncomment when ready
with arcpy.da.InsertCursor(stage_vegetation, field_names) as cursor:
    for row in data:
        cursor.insertRow([row[field] for field in field_names])


### SEZ data from 2020 threshold

In [ ]:
#All SEZ Scores from current Data

# Path to the geodatabase
master_path = r"F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data\SEZ_Data.gdb"

# Table name in the geodatabase
table_name = "AssessmentUnits_Master"

# Function to convert geodatabase table to DataFrame
def table_to_dataframe(table):
    fields = [field.name for field in arcpy.ListFields(table)]
    data = [row for row in arcpy.da.SearchCursor(table, fields)]
    return pd.DataFrame(data, columns=fields)

try:
    # Load the data from the geodatabase table into a DataFrame
    SEZ_Master = table_to_dataframe(os.path.join(master_path, table_name))

    # Remove leading and trailing whitespace from column names
    SEZ_Master.columns = SEZ_Master.columns.str.strip()

    # Columns to select from the table
    selected_columns = ['Acres',
                        'Final_Percent',
                        'Final_Points_Possible',
                        'Final_Rating',
                        'Final_Total_Points',
                        'SEZ_ID',
                        'Comments',
                        'SEZ_Type', 
                        'Assessment_Unit_Name']

    # Create a new DataFrame with selected columns
    SEZ20_df = SEZ_Master[selected_columns].copy()

except KeyError as e:
    print("One or more specified columns not found in the DataFrame:", e)

except Exception as e:
    print("An error occurred:", e)

In [ ]:
#Prep Data
SEZ20_df['Year'] = 2020

#Field Mapping
field_mapping = {
                'Assessment_Unit_Name': 'Assessment_Unit_Name',
                'Year': 'Year',
                'Acres': 'Acres',
                'SEZ_Type': 'SEZ_Type',                    
                'Final_Percent': 'Final_Percent',
                'Final_Total_Points': 'Final_Total_Points',
                'SEZ_ID': 'SEZ_ID',
                'Final_Points_Possible': 'Final_Points_Possible',
                'Final_Rating': 'Final_Rating',
                'Comments': 'Comments'
}

# Rename fields based on field mappings
readydf = SEZ20_df.rename(columns=field_mapping).drop(columns=[col for col in SEZ20_df.columns if col not in field_mapping])

#readydf['SEZ_ID'] = readydf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(readydf)

# Convert DataFrame to a list of dictionaries
data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(readydf.columns)

# Append data to existing table uncomment when ready
with arcpy.da.InsertCursor(stage_All_SEZ_Scores, field_names) as cursor:
    for row in data:
        cursor.insertRow([row[field] for field in field_names])

print("Data appended to table successfully.")

## Final SEZ Scores Calculations for SEZ Threshold

### Set Up Data with REST SERVICE

In [5]:

#Paths to Staging tables in SDE... via REST service
# Use rest service to get data 
#Setup
import pandas as pd
import os
from arcgis.features import FeatureLayer
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def get_fs_data(fs_url):
    feature_layer = FeatureLayer(fs_url)
    query_result = feature_layer.query()
    feature_list = query_result.features
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    return all_data

def get_fs_data_spatial(fs_url):
    feature_layer = FeatureLayer(fs_url)
    query_result = feature_layer.query().sdf
    query_result.spatial.sr = 26910
    return query_result


#def get_fs_data(SEZ_url):
 #   feature_layer = FeatureLayer(SEZ_url)
  #  query_result = feature_layer.query()
   # feature_list = query_result.features
    #all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    #return all_data

bank_stability_url = "https://maps.trpa.org/server/rest/services/SEZ_Assessment_Unit/FeatureServer/4"
biotic_integrity_url = "https://maps.trpa.org/server/rest/services/SEZ_Assessment_Unit/FeatureServer/5"
conifer_url = "https://maps.trpa.org/server/rest/services/SEZ_Assessment_Unit/FeatureServer/6"
ditches_url = "https://maps.trpa.org/server/rest/services/SEZ_Assessment_Unit/FeatureServer/7"
invasives_url = "https://maps.trpa.org/server/rest/services/SEZ_Assessment_Unit/FeatureServer/11"
Hab_Frag_url = 'https://maps.trpa.org/server/rest/services/SEZ_Assessment_Unit/FeatureServer/8'
vegetation_url = "https://maps.trpa.org/server/rest/services/SEZ_Assessment_Unit/FeatureServer/12"
incision_url = "https://maps.trpa.org/server/rest/services/SEZ_Assessment_Unit/FeatureServer/10"
headcuts_url = "https://maps.trpa.org/server/rest/services/SEZ_Assessment_Unit/FeatureServer/9"
AOP_url= "https://maps.trpa.org/server/rest/services/SEZ_Assessment_Unit/FeatureServer/3"
#all_sez_scores_url = ""

SEZ_url = "https://maps.trpa.org/server/rest/services/SEZ_Assessment_Unit/FeatureServer/0"




#Where data will be appended
#All_SEZ_Scores_df = table_to_dataframe(os.path.join(master_path, "All_SEZ_Scores"))

#Where do the comments come from??? Maybe SDE Collect in SEZ Site information? sde.sez_survey



In [6]:
#Create dataframes from Rest Services

dfbanks = get_fs_data(bank_stability_url)
dfbiotic = get_fs_data(biotic_integrity_url)
dfconifer = get_fs_data(conifer_url)
dfditch = get_fs_data(ditches_url)
dfinvasive = get_fs_data(invasives_url)
dfhabitat = get_fs_data(Hab_Frag_url)
dfvegetation = get_fs_data(vegetation_url)
dfincision = get_fs_data(incision_url)
dfheadcuts = get_fs_data(headcuts_url)
dfAOP = get_fs_data(AOP_url)
dfSEZ = get_fs_data_spatial(SEZ_url)





### Prep data in staging tables

In [17]:
# #------------------#
#Biotic Integrity
#------------------#
#Prep data- Add any scores and find average oif there are two stream sites for one sez. Also rename data source so it includes are streams that were averaged
# Function to average scores and concatenate data sources for each Year and Assessment_Unit_Name
def average_biotic_scores(dfbiotic, unit_col='Assessment_Unit_Name', year_col='Year', score='Biotic_Integrity_CSCI', source_col='Biotic_Integrity_Data_Source'):
    # Group by Assessment Unit and Year
    group = dfbiotic.groupby([unit_col, year_col])
    
    # Calculate the mean of the scores
    averaged_scores = group[score].mean().reset_index()
    
    # Concatenate the data sources with specific formatting
    def concatenate_sources(x, year):
        formatted_sources = []
        for entry in x:
            parts = entry.split(",")
            if len(parts) >= 3:
                formatted_sources.append(f'TRPA, {parts[1].strip()}, {parts[-1].strip()}')  # Extract station code and year
        if formatted_sources:
            return '/ '.join(formatted_sources)
        else:
            return None  # Return None if all entries are invalid
    
    # Apply concatenate_sources to each group
    concatenated_sources = group.apply(lambda grp: concatenate_sources(grp[source_col], grp[year_col])).reset_index(name=source_col)
    
    # Merge the averaged scores with concatenated sources
    averaged_df = pd.merge(averaged_scores, concatenated_sources, on=[unit_col, year_col], how='left')
    
    return averaged_df


# Drop duplicates based on 'Assessment_Unit_Name' and 'Year'
dfbiotic = dfbiotic.drop_duplicates(subset=['Assessment_Unit_Name', 'Year', 'Biotic_Integrity_CSCI'])

# Apply the function to dfbiotic
averaged_biotic_df = average_biotic_scores(dfbiotic)

# Apply the rating function to the averaged biotic integrity scores
averaged_biotic_df['Biotic_Integrity_Rating'] = averaged_biotic_df['Biotic_Integrity_CSCI'].apply(categorize_csci)

# Calculate the biotic score for each SEZ
averaged_biotic_df['Biotic_Integrity_Score'] = averaged_biotic_df['Biotic_Integrity_Rating'].apply(score_indicator)

averaged_biotic_df['Biotic_Integrity_Score']=averaged_biotic_df['Biotic_Integrity_Score'].astype(int)

# Output the resulting DataFrame
print(averaged_biotic_df)


         Assessment_Unit_Name  Year  Biotic_Integrity_CSCI  \
0    Angora Creek - tributary  2013               0.996000   
1        Angora Creek - upper  2013               0.996000   
2          Angora meadows - 1  2019               0.690000   
3          Angora meadows - 2  2019               0.820000   
4          Angora meadows - 3  2017               0.940000   
..                        ...   ...                    ...   
226       Woods Creek - lower  2018               0.874000   
227      Woods Creek - middle  2015               1.014000   
228       Woods Creek - upper  2010               1.101000   
229            small meadow 1  2018               1.010000   
230           small meadow 57  2022               0.564053   

    Biotic_Integrity_Data_Source Biotic_Integrity_Rating  \
0          TRPA, 634S13217, 2013                       A   
1          TRPA, 634S13217, 2013                       A   
2          TRPA, 634S19606, 2019                       C   
3          TRPA

In [8]:
#-------------------
#Headcuts 
#------------------
#Reorganize dfHeadcuts to drop small medium large headcut columns
# Drop the columns 'small', 'medium', and 'large'
dfheadcuts = dfheadcuts.drop(columns=['small', 'medium', 'large'])

# Print the DataFrame to see the changes
print(dfheadcuts)


     OBJECTID  SEZ_ID          Assessment_Unit_Name  Year  Number_of_Headcuts  \
0           1   519.0      Angora Creek - tributary  2023                   4   
1           2   144.0            Angora meadows - 5  2019                   4   
2           5   217.0  Angora meadows tributary - 1  2019                   1   
3           6    99.0  Angora meadows tributary - 2  2019                   7   
4           7    99.0  Angora meadows tributary - 2  2023                   4   
..        ...     ...                           ...   ...                 ...   
341       444    44.0               small meadow 96  2019                   0   
342       445    62.0               small meadow 98  2019                   0   
343       446    75.0               small meadow 99  2020                   0   
344       447    75.0               small meadow 99  2022                   0   
345       448     NaN                unnamed meadow  2019                   0   

    Headcuts_Data_Source  H

In [9]:

#Prep SEZ Baseline Data for assessment unit.. does this make the polygons roll over? how do i do that ==make spatially enabled dataframe and add 'SHAPE'
keep_columns = ['SHAPE', 'SEZ_ID', 'Feature_Type', 'SEZ_Type', 'Ownership_Primary', 'Ownership_Secondary', 'Ownership_Secondary_2', 'Ownership_Secondary_3', 'Acres', 'Comments']

dfSEZinfo=dfSEZ.loc[:,keep_columns].copy()

dfSEZinfo['SEZ_ID']= dfSEZinfo['SEZ_ID'].astype(int)

In [10]:
#add year to data source so we can drop the year column later

#Create Dictionary of Dataframes to adjust year to be in datashource column and not its own column
yeartodatasource = {
    'dfbanks': dfbanks,
    'dfheadcuts': dfheadcuts,
    'dfincision': dfincision,
    'dfinvasive': dfinvasive
}

# Iterate over each DataFrame in meadowdata
for name, df in yeartodatasource.items():
    # Iterate over columns in the DataFrame
    for col in df.columns:
        # Check if the column name contains 'Data'
        if 'Data_' in col:
            # Add Year to the column if it contains 'Data'
            df[col] = df[col] + ', ' + df['Year'].astype(str)

### Don't USE This is for All SEZ Scores Only no othe rdata.. DONT USE unless need all _sez_Score table with just scores of Indicator and SEZ no floof

In [ ]:
# DONT USECreate RiverineIndicators list containing specific dataframes
RiverineIndicators = ['Assessment_Unit_Name', 'AOP_Score', 'Bank_Stability_Score', 'Biotic_Integrity_Score', 'Habitat_Frag_Score', 'Incision_Score', 'Headcuts_Score']

mergedriverine_df= mergedmeadow_df[RiverineIndicators]

print(mergedriverine_df)

In [ ]:
#DONT USECalculate Scores for Each SEZ This will go into table All_SEZ_Scores

#FOR MEADOWS


#mergedmeadow_df['SEZ_ID']=mergedmeadow_df['Assessment_Unit_Name'].map(lookup_dict)
# Identify columns that contain 'score' in their names
score_columns = [col for col in mergedmeadow_df.columns if 'Score' in col]

# Sum the values across identified score columns to get the final points for the SEZ
mergedmeadow_df['Final_Total_Points'] = mergedmeadow_df[score_columns].sum(axis=1, skipna=True)

#POints Possible per SEZ

mergedmeadow_df['Final_Points_Possible'] = mergedmeadow_df[score_columns].notna().sum(axis=1)*12


#ASsign Meadow SEZ_ID to each Assessment_Unit

mergedmeadow_df['SEZ_ID']=mergedmeadow_df['Assessment_Unit_Name'].map(lookup_dict)

#FOR QA purposes add SEZ_type so we can make sure lal data is available
mergedmeadow_df['SEZ_Type'] = dfSEZ['SEZ_Type']
#Would it be better to just add up how many indicators have a score? 12 x5?
#mergedmeadow_df['Final_Points_Possible']= dfSEZ['Final_Points_Possible']

# Merge based on 'SEZID'
#mergedmeadow_df = pd.merge(mergedmeadow_df, dfSEZ[['SEZ_ID', 'Final_Points_Possible']], on='SEZ_ID', how='left')

# Assign 'Final_Points_Possible' from dfSEZ to mergedmeadow_df
#mergedmeadow_df['Final_Points_Possible'] = mergedmeadow_df['Final_Points_Possible']
#or? just base of of how many indicators are not null for each row?
#mergedmeadow_df['Final_Points_Possible2']= (12 x number of columns per row that say score have data?) 



#--------------------------------------
# Calculate the final rating
#------------------------------------
mergedmeadow_df['Final_Percent'] = mergedmeadow_df['Final_Total_Points'] / mergedmeadow_df['Final_Points_Possible']

mergedmeadow_df['Final_Rating']= mergedmeadow_df['Final_Percent'].apply(rate_SEZ)

mergedmeadow_df['Final_Score']= mergedmeadow_df['Final_Rating'].apply(score_indicator)






In [ ]:
#DONT USECalculate Scores for Each SEZ This will go into table All_SEZ_Scores

#FOR RIVERINE SEZ
# Identify columns that contain 'score' in their names
score_columns = [col for col in mergedriverine_df.columns if 'Score' in col]

# Sum the values across identified score columns to get the final points for the SEZ
mergedriverine_df['Final_Total_Points'] = mergedriverine_df[score_columns].sum(axis=1, skipna=True)

#POints Possible per SEZ

mergedriverine_df['Final_Points_Possible'] = mergedriverine_df[score_columns].notna().sum(axis=1)*12


#ASsigndriverine SEZ_ID to each Assessment_Unit

mergedriverine_df['SEZ_ID']=mergedriverine_df['Assessment_Unit_Name'].map(lookup_riverine)

#FOR QA purposes add SEZ_type so we can make sure lal data is available
mergedriverine_df['SEZ_Type'] = dfSEZ['SEZ_Type']

#--------------------------------------
# Calculate the final rating
#------------------------------------
mergedriverine_df['Final_Percent'] = mergedriverine_df['Final_Total_Points'] / mergedriverine_df['Final_Points_Possible']

mergedriverine_df['Final_Rating']= mergedriverine_df['Final_Percent'].apply(rate_SEZ)

mergedriverine_df['Final_Score']= mergedriverine_df['Final_Rating'].apply(score_indicator)



In [ ]:
#DONT USEMerge Riverine and Meadow dataframes for a final dataframe

# Concatenate DataFrames doesn't work-try merging--Do we want to make an All_sez_scores
both_df = pd.concat([mergedriverine_df, mergedmeadow_df], axis=0, ignore_index=True)




In [ ]:
#NOTS WORKINNG... WILL NEED THIS for FINAL TABLEsezsurveytable grab comments to add to dataframe--MAYBE JUST DO THIS MANUALLY
#Convert feature class in gdb to Pandas DataFrame this is for REST Service
sezsurveyfields = ['Assessment_Unit_Name', 'Comments', 'Survey_Date']

# Initialize an empty list to store data
data = []

# Iterate over the rows in the feature class and extract data
with arcpy.da.SearchCursor(sezsurveytable, sezsurveyfields) as cursor:
    for row in cursor:
        data.append(row)

sezsurvey_df= pd.DataFrame(data, columns=sezsurveyfields)

sezsurvey_df['Year'] = sezsurvey_df['Survey_Date'].dt.year
print(sezsurvey_df)
#add comments to dataframe
# Create a dictionary to map Assessment_Unit_Name to Comments
comments_map = dict(zip(sezsurvey_df['Assessment_Unit_Name'], sezsurvey_df['Comments']))

# Add the Comments column to both_df using the map function
both_df['Comments'] = both_df['Assessment_Unit_Name'].map(comments_map)




In [ ]:

#DONT USE
# both_df['Year']= '2024'

both_df = both_df.dropna(subset='SEZ_ID')

#----------------------------------------------------------------#
#Prep and post ending dataframe to All Scores table in SEZ_Data.GDB
#----------------------------------------------------------------#
#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'Year': 'Year',
    'Comments': 'Comments',
    'SEZ_Type': 'SEZ_Type',
    'Final_Rating': 'Final_Rating',
    'Final_Percent': 'Final_Percent',
    'SEZ_ID': 'SEZ_ID',
    'Final_Points_Possible': 'Final_Points_Possible',
    'Final_Total_Points': 'Final_Total_Points'
}

# Rename fields based on field mappings
readydf = both_df.rename(columns=field_mapping).drop(columns=[col for col in both_df.columns if col not in field_mapping])

#readydf['SEZ_ID'] = readydf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(readydf)

# Assuming ready_df is the DataFrame you want to write to CSV
readydf.to_csv('ready_df.csv', index=False)

allscoresdf=readydf
# Convert DataFrame to a list of dictionaries
#data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
#field_names = list(readydf.columns)

# Append data to existing table
#with arcpy.da.InsertCursor(allsezscores, field_names) as cursor:
 #   for row in data:
  #      cursor.insertRow([row[field] for field in field_names])

#print("Data appended to table successfully.")

### SDE.SEZ Assessment_Unit final table with all info

In [18]:
#Clean Data, Assign SEZ's based on polygon size, get most recent
#Take df's and create a riverine and meadow df using small polygon size and large polygon size
from functools import reduce
import pandas as pd

# Function to Get most recent year of data from each DataFrame
def get_most_recent_scores(df, groupfield):
    df = df.loc[df.groupby(groupfield)['Year'].idxmax()]
    return df

# Function to merge DataFrames and assign SEZ IDs
def merge_and_assign_sez_ids(cleaned_data, lookup_dict):
    # Merge all DataFrames on 'Assessment_Unit_Name' using reduce
    merged_df = reduce(lambda left, right: pd.merge(left, right, on='Assessment_Unit_Name', how='outer'), cleaned_data.values())
    
    # Assign SEZ_ID based on 'Assessment_Unit_Name' using lookup_dict
    merged_df['SEZ_ID'] = merged_df['Assessment_Unit_Name'].map(lookup_dict)
    
    # Drop rows with missing SEZ_ID
    merged_df = merged_df.dropna(subset=['SEZ_ID'])
    
    merged_df['SEZ_ID'] = merged_df['SEZ_ID'].astype(int)

    return merged_df

# Main function to process data frames, get most recent data, clean, and merge
def get_most_recent_and_clean(meadowdata, riverinedata, lookup_dict, lookup_riverine, columns_to_drop):
    most_recent_data_meadow = {}
    cleaned_data_meadow = {}
    
    most_recent_data_riverine = {}
    cleaned_data_riverine = {}
    
    # Process meadowdata
    for key, df in meadowdata.items():
        processed_df = get_most_recent_scores(df, 'Assessment_Unit_Name')
        most_recent_data_meadow[key] = processed_df
        
        # Drop specified columns and remove duplicate rows
        df_cleaned = processed_df.drop(columns=[col for col in columns_to_drop if col in processed_df.columns])
        df_cleaned = df_cleaned.drop_duplicates()
        cleaned_data_meadow[key] = df_cleaned
    
    # Process riverinedata
    for key, df in riverinedata.items():
        processed_df = get_most_recent_scores(df, 'Assessment_Unit_Name')
        most_recent_data_riverine[key] = processed_df
        
        # Drop specified columns and remove duplicate rows
        df_cleaned = processed_df.drop(columns=[col for col in columns_to_drop if col in processed_df.columns])
        df_cleaned = df_cleaned.drop_duplicates()
        cleaned_data_riverine[key] = df_cleaned
    
    # Merge and assign SEZ IDs for meadowdata
    merged_meadow_df = merge_and_assign_sez_ids(cleaned_data_meadow, lookup_dict)
    
    # Merge and assign SEZ IDs for riverinedata
    merged_riverine_df = merge_and_assign_sez_ids(cleaned_data_riverine, lookup_riverine)
    
    # Combine merged_meadow_df and merged_riverine_df
    combined_df = pd.concat([merged_meadow_df, merged_riverine_df], axis=0, join='outer')

    return {
        'merged_meadow_df': merged_meadow_df,
        'merged_riverine_df': merged_riverine_df,
        'combined_df': combined_df,
        'cleaned_data_meadow': cleaned_data_meadow,
        'cleaned_data_riverine': cleaned_data_riverine,
        'most_recent_data_meadow': most_recent_data_meadow,
        'most_recent_data_riverine': most_recent_data_riverine
    }


In [19]:
#Same for meadow and riverine data drop these columns because not needed in final merge, will assign SEZ ID later
columns_to_drop = {'Year', 'SEZ_ID', 'GlobalID', 'last_edited_user', 'created_date', 'OBJECTID', 'created_user', 'last_edited_date'}

#Name dataframes so we can reference later
meadowdata= {'dfbanks': dfbanks, 
             'dfaveraged_biotic':averaged_biotic_df,
                'dfconifer': dfconifer,
                'dfditch': dfditch,
                'dfinvasive': dfinvasive,
                'dfhabitat': dfhabitat,
                'dfvegetation': dfvegetation,
                'dfincision': dfincision,
                'dfheadcuts': dfheadcuts,
                'dfAOP': dfAOP
}


#Staging Tables Riverine/ small polygons
riverinedata = {'dfheadcuts': dfbanks,
                'dfbiotic': averaged_biotic_df,
                'dfhabitat': dfhabitat,
                'dfincision': dfincision,
                'dfheadcuts': dfheadcuts,
                'dfAOP': dfAOP
}

In [20]:
# Call the function with all the required arguments
result = get_most_recent_and_clean(meadowdata, riverinedata, lookup_dict, lookup_riverine, columns_to_drop)

# Access the results
merged_meadow_df = result['merged_meadow_df']
merged_riverine_df = result['merged_riverine_df']
combined_df=result['combined_df']
cleaned_data_meadow = result['cleaned_data_meadow']
cleaned_data_riverine = result['cleaned_data_riverine']
most_recent_data_meadow = result['most_recent_data_meadow']
most_recent_data_riverine = result['most_recent_data_riverine']

In [21]:
# Join SEZinfo to combined df to get a dataframe with more info about each Assessment Unit

# Join SEZinfo to the combined_df using SEZ_ID
final_df = pd.merge(combined_df, dfSEZinfo, on='SEZ_ID')

#Assign Threshold Calculations which is the Threshold Year--> is just the most recent data within the past 4 years
final_df['Threshold_Year'] = '2023'

In [15]:
# Check the data type of the SEZ_ID column in combined_df
print("Data type of SEZ_ID in combined_df:", combined_df['SEZ_ID'].dtype)

# Check the data type of the SEZ_ID column in dfSEZinfo
print("Data type of SEZ_ID in dfSEZinfo:", dfSEZinfo['SEZ_ID'].dtype)

Data type of SEZ_ID in combined_df: int32
Data type of SEZ_ID in dfSEZinfo: int32


In [27]:
score_columns = [col for col in final_df.columns if 'Score' in col]
print("score columns in final_df:", final_df[score_columns].columns)

score columns in final_df: Index(['Bank_Stability_Score', 'Biotic_Integrity_Score',
       'Conifer_Encroachment_Score', 'Ditches_Score', 'Invasives_Scores',
       'Habitat_Frag_Score', 'VegetationVigor_Score', 'Incision_Score',
       'Headcuts_Score', 'AOP_Score', 'Final_Score'],
      dtype='object')


In [71]:
##RUN FOR QA so you can see the resulting dataframe/scores/formatting etc.
#Will need to rerun to fix dropping the shape column
# List of columns to export (excluding 'geometry' or 'Shape')
columns_to_export = [col for col in final_df.columns if col != 'SHAPE']
final_df[columns_to_export].to_csv('ready_df.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'ready_df.csv'

In [22]:
#Calculate Score based on SEZ_Type
#Use SEZ_Type to select only needed indicators for SEZ Type

# Define the score columns needed for each SEZ Type
RiverineIndicators = ['AOP_Score', 'Bank_Stability_Score', 'Biotic_Integrity_Score', 'Habitat_Frag_Score', 'Incision_Score', 'Headcuts_Score']
ForestRiverineIndicators =['AOP_Score', 'Bank_Stability_Score', 'Biotic_Integrity_Score', 'Invasives_Scores', 'Ditches_Score', 'Habitat_Frag_Score', 'Incision_Score', 'Headcuts_Score']
NonChanneledIndicators=['Invasives_Scores', 'Conifer_Encroachment_Score', 'Ditches_Score', 'Habitat_Frag_Score', 'Headcuts_Score', 'VegetationVigor_Score']
ChanneledIndicators= ['AOP_Score', 'Bank_Stability_Score', 'Biotic_Integrity_Score', 'Invasives_Scores', 'Conifer_Encroachment_Score', 'Ditches_Score', 'Habitat_Frag_Score', 'Incision_Score', 'Headcuts_Score', 'VegetationVigor_Score']
ForestIndicators= ['Bank_Stability_Score', 'Ditches_Score', 'Habitat_Frag_Score', 'Headcuts_Score']

# Function to get the score columns based on SEZ_Type
def get_score_columns(sez_type):
    if sez_type == 'Riverine (Perennial)':
        return RiverineIndicators
    elif sez_type == 'Riverine (Perennial) + Forested':
        return ForestRiverineIndicators
    elif sez_type == 'Non-Channeled Meadow':
        return NonChanneledIndicators
    elif sez_type == 'Channeled Meadow':
        return ChanneledIndicators
    elif sez_type == 'Forested':
        return ForestIndicators
    else:
        return []

# Apply the appropriate score columns based on SEZ_Type
final_df['Score_Columns'] = final_df['SEZ_Type'].apply(get_score_columns)

# Function to calculate the final points and points possible
def calculate_scores(row):
    score_columns = row['Score_Columns']
    if not score_columns:
        return pd.Series([None, None])
    total_points = row[score_columns].sum(skipna=True)
    points_possible = row[score_columns].notna().sum() * 12
    return pd.Series([total_points, points_possible])

# Apply the score calculation to each row
final_df[['Final_Total_Points', 'Final_Points_Possible']] = final_df.apply(calculate_scores, axis=1)

# Calculate the final percent
final_df['Final_Percent'] = final_df['Final_Total_Points'] / final_df['Final_Points_Possible']

# Calculate the final rating and score
final_df['Final_Rating'] = final_df['Final_Percent'].apply(rate_SEZ)
final_df['Final_Score'] = final_df['Final_Rating'].apply(score_indicator)

# Drop the temporary 'Score_Columns' column
final_df = final_df.drop(columns=['Score_Columns'])

# Convert SEZ_ID to string
#final_df['SEZ_ID'] = final_df['SEZ_ID'].astype(str)


In [56]:
##RUN FOR QA so you can see the resulting dataframe/scores/formatting etc.
#Will need to rerun to fix dropping the shape column
# List of columns to export (excluding 'geometry' or 'Shape')
columns_to_export = [col for col in final_df.columns if col != 'SHAPE']
final_df[columns_to_export].to_csv('ready_df.csv', index=False)

In [57]:
print(final_df.columns)

Index(['Assessment_Unit_Name', 'Bank_Stability_Data_Source',
       'Bank_Stability_Percent_Unstable', 'Bank_Stability_Rating',
       'Bank_Stability_Score', 'Biotic_Integrity_CSCI',
       'Biotic_Integrity_Data_Source', 'Biotic_Integrity_Rating',
       'Biotic_Integrity_Score', 'Conifer_Encroachment_Data_Sourc',
       'Conifer_Percent_Encroached', 'Conifer_Encroachment_Rating',
       'Conifer_Encroachment_Score', 'ConiferEncroachment_Comments',
       'Ditches_Data_Source', 'Ditches_Length', 'Ditches_Meadow_Length',
       'Ditches_Percent', 'Ditches_Rating', 'Ditches_Score',
       'Invasives_Data_Source', 'Invasives_Number_of_Invasives',
       'Invasive_Percent_Cover', 'Invasives_Plant_Types', 'Invasives_Rating',
       'Invasives_Scores', 'Habitat_Frag_Data_Source',
       'Habitat_Frag_Impervious_Acres', 'Habitat_Frag_Percent_Impervious',
       'Habitat_Frag_Rating', 'Habitat_Frag_Score',
       'VegetationVigor_DataSource', 'VegetationVigor_Rating',
       'VegetationVigor

In [23]:
#Final set up


#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'SHAPE': 'SHAPE',
    'Threshold_Year': 'Threshold Year',
    'Comments': 'Comments',
    'SEZ_Type': 'SEZ_Type',
    'Final_Rating': 'Final_Rating',
    'Final_Percent': 'Final_Percent',
    'SEZ_ID': 'SEZ_ID',
    'Final_Points_Possible': 'Final_Points_Possible',
    'Final_Total_Points': 'Final_Total_Points',
    'Acres': 'Acres',
    'AOP_BarriersPerMile':'AquaticOrganismPassage_Barriers',
    'AOP_NumberofBarriers': 'AquaticOrganismPassage_NumberOf',
    'AOP_Score': 'AquaticOrganismPassage_Score',
    'AOP_Rating': 'AquaticOrganismPassage_Rating',
    'AOP_StreamMiles': 'AquaticOrganismPassage_StreamMiles',
    'AOP_DataSource': 'AquaticOrganismPassage_DataSour',
    'Bank_Stability_Data_Source': 'Bank_Stability_Data_Source',
    'Bank_Stability_Percent_Unstable': 'Bank_Stability_Percent_Unstable',
    'Bank_Stability_Rating': 'Bank_Stability_Rating',
    'Bank_Stability_Score': 'Bank_Stability_Score',
    'Biotic_Integrity_Rating': 'Biotic_Integrity_Rating',
    'Biotic_Integrity_CSCI': 'Biotic_Integrity_CSCI',
    'Biotic_Integrity_Data_Source': 'Biotic_Integrity_Data_Source',
    'Biotic_Integrity_Score': 'Biotic_Integrity_Score',
    'Comments': 'Comments',
    'Conifer_Percent_Encroached': 'Conifer_Encroachment_Percent_En',
    'Conifer_Encroachment_Data_Sourc': 'Conifer_Encroachment_Data_Sourc',
    'Conifer_Encroachment_Rating': 'Conifer_Encroachment_Rating',
    'Conifer_Encroachment_Score': 'Conifer_Encroachment_Score',
    'ConiferEncroachment_Comments': 'Conifer_Encroachment_Comments',
    'Ditches_Data_Source': 'Ditches_Data_Source',
    'Ditches_Length': 'Ditches_Length',
    'Ditches_Meadow_Length': 'Ditches_Meadow_Length',
    'Ditches_Percent': 'Ditches_Percent',
    'Ditches_Rating': 'Ditches_Rating',
    'Ditches_Score': 'Ditches_Score',
    'Feature_Type': 'Feature_Type',
    'Habitat_Frag_Data_Source': 'Habitat_Fragmentation_Data_Sour',
    'Habitat_Frag_Impervious_Acres': 'Habitat_Fragmentation_Imperviou',
    'Habitat_Frag_Percent_Impervious': 'Habitat_Fragmentation_Percent_I',
    'Habitat_Frag_Rating': 'Habitat_Fragmentation_Rating',
    'Habitat_Frag_Score': 'Habitat_Fragmentation_Score',
    'Headcuts_Data_Source': 'Headcuts_Data_Source',
    'Number_of_Headcuts': 'Headcuts_Number_of_Headcuts',
    'Headcuts_Rating': 'Headcuts_Rating',
    'Headcuts_Score': 'Headcuts_Score',
    'Incision_Data_Source': 'Incision_Data_Source',
    'Incision_Rating': 'Incision_Rating',
    'Incision_Score': 'Incision_Score',
    'Incision_Ratio': 'Incision_Ratio',
    'Invasive_Percent_Cover': 'Invasive_Percent_Cover',
    'Invasives_Rating': 'Invasive_Rating',
    'Invasives_Data_Source': 'Invasives_Data_Source',
    'Invasives_Number_of_Invasives': 'Invasives_Number_of_Invasives',
    'Invasives_Plant_Types': 'Invasives_Plant_Types',
    'Invasives_Scores': 'Invasives_Scores',
    'NDVI_ID': 'NDVI_ID',
    'Ownership_Primary': 'Ownership_Primary',
    'Ownership_Secondary': 'Ownership_Secondary',
    'Ownership_Secondary_2': 'Ownership_Secondary_2',
    'Ownership_Secondary_3': 'Ownership_Secondary_3',
    'VegetationVigor_DataSource': 'VegetationVigor_DataSource',
    'VegetationVigor_Rating': 'VegetationVigor_Rating',
    'VegetationVigor_Raw': 'VegetationVigor_Raw',
    'VegetationVigor_Score': 'VegetationVigor_Score',
}
# Rename fields based on field mappings
SEZscores_readydf = final_df.rename(columns=field_mapping).drop(columns=[col for col in final_df.columns if col not in field_mapping])

#readydf['SEZ_ID'] = readydf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(SEZscores_readydf)

#Post results to CSV in gis/projects/Researchanalysis/SEZ for further QA
columns_to_export = [col for col in SEZscores_readydf.columns if col != 'SHAPE']
#Store csv on F drive for QA/add comments manually on F drive and to change up comments later based on SEZ's that scores changed
final_results = "F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Threshold24_SEZScores.csv"
# Assuming ready_df is the DataFrame you want to write to CSV
SEZscores_readydf[columns_to_export].to_csv(final_results, index=False)


# Convert DataFrame to a list of dictionaries
#data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
#field_names = list(readydf.columns)

# Append data to existing table
#with arcpy.da.InsertCursor(allsezscores, field_names) as cursor:
 #   for row in data:
  #      cursor.insertRow([row[field] for field in field_names])

#print("Data appended to table successfully.")

             Assessment_Unit_Name Bank_Stability_Data_Source  \
0        Angora Creek - tributary                 TRPA, 2023   
1              Angora meadows - 1                 TRPA, 2019   
2              Angora meadows - 2                 TRPA, 2019   
3              Angora meadows - 3                 TRPA, 2022   
4              Angora meadows - 6                 TRPA, 2019   
..                            ...                        ...   
637   Secret Harbor Creek - lower                        NaN   
638  Slaughterhouse Creek - upper                        NaN   
639       Snow Creek wetlands - 1                        NaN   
640         Third Creek - upper 3                        NaN   
641               small meadow 50                        NaN   

     Bank_Stability_Percent_Unstable Bank_Stability_Rating  \
0                           0.859340                     A   
1                           0.000000                     A   
2                           0.000000         

### QA tools

In [ ]:
#Before Final comparison do more QA
    # Check to make sure there are 643 ASsessment Units
    #Check Final Points and make sure there are no really low numbers could be missing data
    #Check all staging tables for completeness- There are ratings A-D, check on null data for that SEZ
    #other QA methods can be added to this list

#Check to see which SEZ Scores Changed
Threshold24sezdata= SEZscores_readydf
Threshold24sezdata =


In [ ]:
#QA only
# DONT USE unless want to use for QA Make a column for QA for SEZ_Type/final points possible
def categorize_finalpoints(SEZ_Type):
     if pd.isna(SEZ_Type):
        return np.nan
     elif   SEZ_Type == 'Non-Channeled Meadow':
        return '72'
     elif SEZ_Type == 'Channeled Meadow':
        return '120'
     elif SEZ_Type == 'Riverine (Perennial)':
        return 'C'
     elif SEZ_Type == 'Forested':
        return '48'
     elif SEZ_Type == 'Riverine (Perennial) + Forested':
        return '96'
     
final_df['TESTFinalPointsPossible']= final_df['SEZ_Type'].apply(categorize_finalpoints)
   

### Orginal code in case we need it--will probably delete

In [ ]:
#Don't USeoriginal code.. merge needs work it seems like it is using lookup dict for both

from functools import reduce
# Functino to Get most recent year of data from each Dataframe
def get_most_recent_scores(df,groupfield):
    df = df.loc[df.groupby(groupfield)['Year'].idxmax()]
    return df

def get_most_recent_and_clean(data_frames, lookup_dict, columns_to_drop):
    most_recent_data = {}

    # Iterate over the items in the original 'meadowdata' dictionary
    for key, df in data_frames.items():
        # Apply the 'get_most_recent_scores' function to each DataFrame
        processed_df = get_most_recent_scores(df, 'Assessment_Unit_Name')
        
        # Store the processed DataFrame in the new dictionary using the same key
        most_recent_data[key] = processed_df
    
# Columns to drop

    # Drop specified columns and remove duplicate rows
    cleaned_data = {}
    for key, df in most_recent_data.items():
        # Drop specified columns if they exist
        df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
        
        # Remove duplicate rows
        df = df.drop_duplicates()
        
        # Append the cleaned DataFrame to the list
        cleaned_data[key] = df
    
    # Merge all DataFrames on 'Assessment_Unit_Name' using reduce
    merged_df = {}
    for key, df in cleaned_data.items():
        merged_df = reduce(lambda left, right: pd.merge(left, right, on='Assessment_Unit_Name', how='outer'), cleaned_data.values())


    # Meadowdata gets the lookup_dict
    # Riverinedatagets lookup_riverine dictionary how the hell do i do this 

    #Add large polygon/meadow feature type sez id
    merged_df['SEZ_ID']=merged_df['Assessment_Unit_Name'].map(lookup_dict)

    merged_df = merged_df.dropna(subset=['SEZ_ID'])

    merged_df[key] = df
    
    return {'merged_df':merged_df,
            'cleaned_data':cleaned_data,
            'most_recent_data':most_recent_data}


In [ ]:
#DONT USE FOR NOW --maybe for final points.. Try SEZ_Type to say what indicators are.. so it would be Foresteddata, Forested+ Riverine(perennial), Riverine(Perennial), Channeled MEadow, Non_Channeled meadow
#MIGHT NOT USE
#Forest and Riverine Indicators
#Name dataframes so we can reference later
forestriverine= {'dfbanks': dfbanks, 
                    'dfaveraged_biotic':averaged_biotic_df,
                    'dfditch': dfditch,
                    'dfinvasive': dfinvasive,
                    'dfhabitat': dfhabitat,
                    'dfincision': dfincision,
                    'dfheadcuts': dfheadcuts,
                    'dfAOP': dfAOP
}
                
#Forest Indicators
forest= {'dfbanks': dfbanks, 
            'dfditch': dfditch,
            'dfhabitat': dfhabitat,
            'dfheadcuts': dfheadcuts,
}
#Channeled Meadow Indicators
channeled= {'dfbanks': dfbanks, 
             'dfaveraged_biotic':averaged_biotic_df,
                'dfconifer': dfconifer,
                'dfditch': dfditch,
                'dfinvasive': dfinvasive,
                'dfhabitat': dfhabitat,
                'dfvegetation': dfvegetation,
                'dfincision': dfincision,
                'dfheadcuts': dfheadcuts,
                'dfAOP': dfAOP
}


#Non-Channeled Meadow Indicators
NonChanneled= {'dfconifer': dfconifer,
                'dfditch': dfditch,
                'dfinvasive': dfinvasive,
                'dfhabitat': dfhabitat,
                'dfvegetation': dfvegetation,
                'dfheadcuts': dfheadcuts
}

In [ ]:
#Calculate Scores for Each SEZ This will go into table All_SEZ_Scores

#FOR MEADOWS


#mergedmeadow_df['SEZ_ID']=mergedmeadow_df['Assessment_Unit_Name'].map(lookup_dict)
# Identify columns that contain 'score' in their names
mergedmeadow_df = meadowdata_processed['merged_df']
score_columns = [col for col in mergedmeadow_df.columns if 'Score' in col]

# Sum the values across identified score columns to get the final points for the SEZ
mergedmeadow_df['Final_Total_Points'] = mergedmeadow_df[score_columns].sum(axis=1, skipna=True)

#POints Possible per SEZ

mergedmeadow_df['Final_Points_Possible'] = mergedmeadow_df[score_columns].notna().sum(axis=1)*12


#ASsign Meadow SEZ_ID to each Assessment_Unit

mergedmeadow_df['SEZ_ID']=mergedmeadow_df['Assessment_Unit_Name'].map(lookup_dict)

mergedmeadow_df['SEZ_ID'] = mergedmeadow_df['SEZ_ID'].astype(str)

#--------------------------------------
# Calculate the final rating
#------------------------------------
mergedmeadow_df['Final_Percent'] = mergedmeadow_df['Final_Total_Points'] / mergedmeadow_df['Final_Points_Possible']

mergedmeadow_df['Final_Rating']= mergedmeadow_df['Final_Percent'].apply(rate_SEZ)

mergedmeadow_df['Final_Score']= mergedmeadow_df['Final_Rating'].apply(score_indicator)

In [ ]:
#Calculate Scores for Each SEZ This will go into final table

#FOR RIVERINE SEZ
# Identify columns that contain 'score' in their names
mergedriverine_df = riverinedata_processed['merged_df']
score_columns = [col for col in mergedriverine_df.columns if 'Score' in col]

# Sum the values across identified score columns to get the final points for the SEZ
mergedriverine_df['Final_Total_Points'] = mergedriverine_df[score_columns].sum(axis=1, skipna=True)

#POints Possible per SEZ

mergedriverine_df['Final_Points_Possible'] = mergedriverine_df[score_columns].notna().sum(axis=1)*12


#ASsigndriverine SEZ_ID to each Assessment_Unit

mergedriverine_df['SEZ_ID']=mergedriverine_df['Assessment_Unit_Name'].map(lookup_riverine)

mergedriverine_df['SEZ_ID'] = mergedriverine_df['SEZ_ID'].astype(int)

#FOR QA purposes add SEZ_type so we can make sure lal data is available
#mergedriverine_df['SEZ_Type'] = dfSEZ['SEZ_Type']

#--------------------------------------
# Calculate the final rating
#------------------------------------
mergedriverine_df['Final_Percent'] = mergedriverine_df['Final_Total_Points'] / mergedriverine_df['Final_Points_Possible']

mergedriverine_df['Final_Rating']= mergedriverine_df['Final_Percent'].apply(rate_SEZ)

mergedriverine_df['Final_Score']= mergedriverine_df['Final_Rating'].apply(score_indicator)

In [ ]:
print(dfSEZ.columns)

In [ ]:
# Concatenate mergedmeadow_df and mergedriverine_df
riverinemeadowdf = pd.concat([mergedmeadow_df, mergedriverine_df], axis=0, ignore_index=True)

#riverinemeadowdf['SEZ_ID']= riverinemeadowdf['SEZ_ID'].astype(int)

In [ ]:
riverinemeadowdf_duplicate_sez = riverinemeadowdf[riverinemeadowdf.duplicated(subset='SEZ_ID', keep=False)]

In [ ]:
#NOTS WORKINNG... WILL NEED THIS for FINAL TABLEsezsurveytable grab comments to add to dataframe--MAYBE JUST DO THIS MANUALLY
#Convert feature class in gdb to Pandas DataFrame this is for REST Service
sezsurveyfields = ['Assessment_Unit_Name', 'Comments', 'Survey_Date']

# Initialize an empty list to store data
data = []

# Iterate over the rows in the feature class and extract data
with arcpy.da.SearchCursor(sezsurveytable, sezsurveyfields) as cursor:
    for row in cursor:
        data.append(row)

sezsurvey_df= pd.DataFrame(data, columns=sezsurveyfields)

sezsurvey_df['Year'] = sezsurvey_df['Survey_Date'].dt.year
print(sezsurvey_df)
#add comments to dataframe
# Create a dictionary to map Assessment_Unit_Name to Comments
comments_map = dict(zip(sezsurvey_df['Assessment_Unit_Name'], sezsurvey_df['Comments']))

# Add the Comments column to both_df using the map function
riverinemeadowdf['Comments'] = riverinemeadowdf['Assessment_Unit_Name'].map(comments_map)


In [ ]:
#Join Base data to riverine and meadow data

Final_mergeddf =pd.merge(dfSEZinfo,riverinemeadowdf, on='SEZ_ID', how='outer', indicator= True)

Final_mergeddf['Year'] = '2024'
Final_mergeddf.to_csv('ready_df.csv', index=False)

In [ ]:
Final_mergeddf['_merge'].value_counts()

In [ ]:
#Final set up


#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'Year': 'Threshold Year',
    'Comments': 'Comments',
    'SEZ_Type': 'SEZ_Type',
    'Final_Rating': 'Final_Rating',
    'Final_Percent': 'Final_Percent',
    'SEZ_ID': 'SEZ_ID',
    'Final_Points_Possible': 'Final_Points_Possible',
    'Final_Total_Points': 'Final_Total_Points',
    'Acres': 'Acres',
    '':'AquaticOrganismPassage_Barriers',
    '': 'AquaticOrganismPassage_NumberOf',
    '': 'AquaticOrganismPassage_Score',
    '': 'AquaticOrganismPassage_StreamMiles',
    '': 'Bank_Stability_Data_Source',
    '': 'Bank_Stability_Percent_Unstable',
    '': 'Bank_Stability_Rating',
    '': 'Bank_Stability_Score',
    '': 'Biotic_Integrity_Rating',
    '': 'Biotic_Integrity_CSCI',
    '': 'Biotic_Integrity_Data_Source',
    '': 'Biotic_Integrity_Score',
    '': 'Comments',
    '': 'Conifer_Encroachment_Percent_En',
    '': 'Conifer_Encroachment_Data_Sourc',
    '': 'Conifer_Encroachment_Rating',
    '': 'Conifer_Encroachment_Score',
    '': 'Conifer_Encroachment_Comments',
    '': 'CountAttachments',
    '': 'Ditches_Data_Source',
    '': 'Ditches_Length',
    '': 'Ditches_Meadow_Length',
    '': 'Ditches_Percent',
    '': 'Ditches_Rating',
    '': 'Ditches_Score',
    '': 'Feature_Type',
    '': 'Habitat_Fragmentation_Data_Sour',
    '': 'Habitat_Fragmentation_Imperviou',
    '': 'Habitat_Fragmentation_Percent_I',
    '': 'Habitat_Fragmentation_Rating',
    '': 'Habitat_Fragmentation_Score',
    '': 'Headcuts_Data_Source',
    '': 'Headcuts_Number_of_Headcuts',
    '': 'Headcuts_Rating',
    '': 'Headcuts_Score',
    '': 'Incision_Data_Source',
    '': 'Incision_Rating',
    '': 'Incision_Score',
    '': 'Incision_Ratio',
    '': 'Invasive_Percent_Cover',
    '': 'Invasive_Rating',
    '': 'Invasives_Data_Source',
    '': 'Invasives_Invasives_Number_of_Invasives',
    '': 'Invasives_Plant_Types',
    '': 'Invasives_Scores',
    '': 'NDVI_ID',
    '': 'Ownership_Primary',
    '': 'Ownership_Secondary',
    '': 'Ownership_Secondary_2',
    '': 'Ownership_Secondary_3',
    '': 'VegetationVigor_DataSource',
    '': 'VegetationVigor_Rating',
    '': 'VegetationVigor_Raw',
    '': 'VegetationVigor_Score',
}
# Rename fields based on field mappings
readydf = both_df.rename(columns=field_mapping).drop(columns=[col for col in both_df.columns if col not in field_mapping])

#readydf['SEZ_ID'] = readydf['Assessment_Unit_Name'].map(lookup_dict)

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(readydf)

# Assuming ready_df is the DataFrame you want to write to CSV
readydf.to_csv('ready_df.csv', index=False)


# Convert DataFrame to a list of dictionaries
#data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
#field_names = list(readydf.columns)

# Append data to existing table
#with arcpy.da.InsertCursor(allsezscores, field_names) as cursor:
 #   for row in data:
  #      cursor.insertRow([row[field] for field in field_names])

#print("Data appended to table successfully.")

In [ ]:
# Print the current working directory
print("Current working directory:", os.getcwd())

# Stream Habitat Condition use mergedriverine_df but add IPI

In [ ]:
#Import IPI Data and add it to mergedriverine_df
IPIfolder = "F:\Research and Analysis\Fisheries\Streams\Bioassessment"
IPI22 = os.path.join(IPIfolder, "2022", "IPI_22.csv")
IPI20 = os.path.join(IPIfolder, "2020", "IPI_20.csv")

#Create IPI Dataframes
IPI22df = pd.read_csv(IPI22)
IPI20df = pd.read_csv(IPI20)

IPI22df['IPIYear']= '2022'
IPI20df['IPIYear']= '2020'
#Merge dataframes into one 
# Concatenate IPIdf1 and IPIdf2 along rows (axis=0)
concatIPI_df = pd.concat([IPI22df, IPI20df], axis=0, ignore_index=True)

#Calculate Scores in IPI
#Code for Grading IPI
#Define Grade for IPI Score - Used only for Stream HAbitat Condition
def categorize_phab(IPI):
     if   IPI >= 0.94:
        return 'A'
     elif 0.83 < IPI < 0.94:
        return 'B'
     elif 0.7 < IPI <= 0.83:
        return 'C'
     else:
        return 'D'

concatIPI_df['IPI_Rating']=concatIPI_df['IPI'].apply(categorize_phab)
concatIPI_df['IPI_Score']= concatIPI_df['IPI_Rating'].apply(score_indicator)



concatIPI_df.head()

columns_to_keep = ['StationCode', 'IPI', 'IPI_Rating', 'IPI_Score', 'IPIYear']

concatIPI_df = concatIPI_df[columns_to_keep]



In [ ]:
#THIS MATCHES THE BIOTIC INTEGRITY SCORES, next code block will do a spatial join with the data

#prep biotic data so that we have a station code to match to so we have an sez assessment unit
dfbiotic['StationCode'] = dfbiotic['Biotic_Integrity_Data_Source'].str.split(',').str[1].str.strip()

sezipi_df = pd.merge(dfbiotic, concatIPI_df, on='StationCode', how='outer')


sezipi_df = sezipi_df.dropna(subset=['IPI'])

# Define a function to get the most recent score for each assessment unit
def get_most_recent_scores(reduced_dfs):
    
# Group by 'Assessment_Unit_Name' and find the row with the maximum 'Year' or do i want an average?
most_recent_IPI = df.loc[df.groupby('Assessment_Unit_Name')['Year'].idxmax()]

#AVerage IPI for sez assessment unit

print(most_recent_IPI)


In [ ]:
#Prep mergedriverine_df calculate stream  maybe match scores to biotic integrity since those will all get phab?
#mergedriverine_df = StationCode so I can match data from IPI df to merged riverine
#mergedmeadow_df = pd.merge(mergedmeadow_df, df, on='Assessment_Unit_Name', how='outer')

##### Code to make it a spatially enabled dataframe

In [ ]:
#code to conver to spatially enable geodataframe
# Create a new DataFrame Final_df with columns based on the field mapping
#df = df.rename(columns=field_mapping)

# Convert DataFrame to Spatially Enabled DataFrame
#sedf = GeoAccessor.from_xy(df, x_column='LONGITUDE', y_column='LATITUDE')

# Convert the SEDF to a feature class without sanitizing columns
#sedf.spatial.to_featureclass(location=os.path.join(workspace, 'PlanAreaNoise_23_Staging'), sanitize_columns=False)